In [ ]:
%load_ext autoreload
%autoreload 2

# Import Required Libraries
Import the necessary pdf2image and OpenAI.

In [ ]:
# Importing required libraries
from fastcore.all import AttrDict
import io
from pdf2image import convert_from_path
from pypdf import PdfReader
import openai
import os
import base64
import requests
from dotenv import load_dotenv
import os

load_dotenv()  # Carga las variables de entorno desde .env

# Load and Convert PDF to Image
Use PyPDF2 pdf2image to load the PDF file and convert it into an image.

In [ ]:
# Load the PDF file
path = '../example_data/1.pdf'

# Open and read the PDF file
reader = PdfReader(path)

# Check if the PDF has more than one page
if len(reader.pages) > 1:
    print("The PDF has more than one page. Only the first page will be converted to an image.")

# Convert the first page of the PDF to an image
images = convert_from_path(path, first_page=1, last_page=1, fmt='jpeg')


In [ ]:
# Crop the image
image = images[0].crop((0, 0, images[0].width, 450))
image

# Call OpenAI ChatGPT API with Vision Model
Use the OpenAI API to call the ChatGPT model with the vision capability.

In [ ]:
buffered = io.BytesIO()
images[0].save(buffered, format="JPEG")
base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
base64_image

In [ ]:
# Load moodle students information
import pandas as pd

# Cargar el CSV de alumnos y grupos
students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
dfs = [ pd.read_csv(filename) for filename in students_info ]

df = pd.concat(dfs, ignore_index=True)

# Puedes opcionalmente limpiar espacios y convertir a mayúsculas para facilitar coincidencias
df["Nombre"] = df["Nombre"].str.strip().str.upper()
df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()

guia_texto = "\n".join(
    f"{row['Nombre']} {row['Apellido(s)']} - Grupo: {row['Grupos']}"
    for _, row in df.iterrows()
)

In [ ]:
#print(guia_texto[:3])
#print(guia_texto[3:])

In [ ]:
api_key = os.environ["OPENAI_API_KEY"] # *
prompt = """Extract the last name (Apellidos in Spanish), the first name (Nombre in Spanish) and the group (Grupo in Spanish)
 from the top of the image. You will find them handwritten after the labels `Apellidos`,  `Nombre` and `Grupo` respectively. The fields of your JSON output will have those exact same label names
 Here is a list of expected students and their groups as a reference: 
 {guia_texto}
 """ # *
model = "gpt-4o"

In [20]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": f'{model}',
  "response_format": { "type": "json_object" },
  "messages": [
    {
        "role": "system", 
        "content": "You are a helpful assistant designed to see an exam and output JSON \
           with the extracted information. You will be given an image of the exam.\
           the default group in case of empty string is extraviado",   
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{prompt}"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [21]:
import json

# Obtener el JSON como string
info_str = response.json()['choices'][0]['message']['content']

# Convertir a diccionario Python
info = json.loads(info_str)

# Si el grupo está vacío, marcarlo como "extraviado"
if not info.get("Grupo"):  # También cubre None y ""
    info["Grupo"] = "extraviado"

# Mostrar resultado
print(info)


{'Apellidos': 'RODRIGUEZ FERNANDEZ', 'Nombre': 'VICTOR', 'Grupo': 'extraviado'}


In [22]:
# Add path to the info dictionary
info['path'] = path

In [23]:
path

'../example_data/1.pdf'

In [24]:
import pandas as pd
import os
import zipfile
from pathlib import Path
import unicodedata
import re

def normalizar_texto(texto):
    """Normaliza texto eliminando acentos y caracteres especiales"""
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(char for char in texto if unicodedata.category(char) != 'Mn')
    texto = texto.upper().strip()
    texto = re.sub(r'[^A-Z0-9\s]', '', texto)
    return texto

def buscar_practica_en_zips(apellidos, nombre, practica_num=3, ruta_data="./../data/"):
    """
    Busca si existe una práctica para un alumno en los archivos ZIP
    
    Args:
        apellidos: Apellidos del alumno
        nombre: Nombre del alumno
        practica_num: Número de práctica (3 o 5)
        ruta_data: Ruta base a la carpeta data
    """
    ruta_practica = Path(ruta_data) / f"Practica{practica_num}"
    
    if not ruta_practica.exists():
        return False
    
    # Normalizar apellidos y nombre
    apellidos_norm = normalizar_texto(apellidos)
    nombre_norm = normalizar_texto(nombre)
    
    # Buscar en todos los archivos ZIP
    for archivo_zip in ruta_practica.glob("*.zip"):
        try:
            with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
                for archivo in zip_ref.namelist():
                    archivo_norm = normalizar_texto(archivo)
                    
                    # Verificar si el archivo contiene apellidos y nombre
                    if apellidos_norm in archivo_norm and nombre_norm in archivo_norm:
                        return True
        except Exception as e:
            continue
    
    return False

def verificar_todas_las_practicas(df, ruta_data="./../data/"):
    """
    Añade columnas de verificación de prácticas al DataFrame existente
    
    Args:
        df: DataFrame con columnas 'Nombre' y 'Apellido(s)'
        ruta_data: Ruta base a la carpeta data
    
    Returns:
        DataFrame modificado con las nuevas columnas
    """
    
    # Crear copias para evitar warnings
    df_resultado = df.copy()
    
    # Inicializar las nuevas columnas
    df_resultado['Presentada_3'] = 0
    df_resultado['Comentario_3'] = 'NP'
    df_resultado['Presentada_5'] = 0
    df_resultado['Comentario_5'] = 'NP'
    
    print("Verificando entregas de prácticas...")
    print("="*50)
    
    practicas_3_encontradas = 0
    practicas_5_encontradas = 0
    total_alumnos = len(df_resultado)
    
    for idx, row in df_resultado.iterrows():
        nombre = str(row['Nombre'])
        apellidos = str(row['Apellido(s)'])
        
        # Verificar Práctica 3
        tiene_practica3 = buscar_practica_en_zips(apellidos, nombre, 3, ruta_data)
        if tiene_practica3:
            df_resultado.loc[idx, 'Presentada_3'] = 1
            df_resultado.loc[idx, 'Comentario_3'] = ''
            practicas_3_encontradas += 1
        
        # Verificar Práctica 5
        tiene_practica5 = buscar_practica_en_zips(apellidos, nombre, 5, ruta_data)
        if tiene_practica5:
            df_resultado.loc[idx, 'Presentada_5'] = 1
            df_resultado.loc[idx, 'Comentario_5'] = ''
            practicas_5_encontradas += 1
        
        # Mostrar progreso
        status_3 = "✓" if tiene_practica3 else "✗"
        status_5 = "✓" if tiene_practica5 else "✗"
        print(f"{status_3} P3 | {status_5} P5 | {apellidos}, {nombre}")
    
    print("="*50)
    print(f"RESUMEN:")
    print(f"Total alumnos verificados: {total_alumnos}")
    print(f"Práctica 3 - Entregadas: {practicas_3_encontradas} | No entregadas: {total_alumnos - practicas_3_encontradas}")
    print(f"Práctica 5 - Entregadas: {practicas_5_encontradas} | No entregadas: {total_alumnos - practicas_5_encontradas}")
    
    return df_resultado

In [25]:
# Load moodle students information
import pandas as pd
# Cargar el CSV de alumnos y grupos
students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
dfs = [ pd.read_csv(filename) for filename in students_info ]
df = pd.concat(dfs, ignore_index=True)
# Limpiar espacios y convertir a mayúsculas
df["Nombre"] = df["Nombre"].str.strip().str.upper()
df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
print("DataFrame original cargado:")
print(f"Total alumnos: {len(df)}")
print(df.head())
print("\n")

# Verificar prácticas y añadir columnas
df_con_practicas = verificar_todas_las_practicas(df)

print("\nDataFrame con verificación de prácticas:")
print(df_con_practicas.head())

# Mostrar estadísticas por grupo si existe la columna
if 'Grupos' in df_con_practicas.columns:
    print("\nEstadísticas por grupo:")
    resumen_grupos = df_con_practicas.groupby('Grupos').agg({
        'Presentada_3': 'sum',
        'Presentada_5': 'sum'
    })
#    print(resumen_grupos)

DataFrame original cargado:
Total alumnos: 444
        Nombre       Apellido(s)                   Dirección de correo  \
0        SOFIA   AGAPITO DELGADO              s.agapito@alumnos.upm.es   
1  LLOYD DAREN    AGUILAR DESIAR          daren.aguilar@alumnos.upm.es   
2       JAVIER   AGUIRRE HERVIAS  javier.aguirrehervias@alumnos.upm.es   
3        MATEO          ALBRIZIO         mateo.albrizio@alumnos.upm.es   
4      NICOLAS  ALONSO FERNANDEZ              ni.alonso@alumnos.upm.es   

    Grupos  
0  IWSIT11  
1  IWSIM12  
2  IWSIM12  
3  IWSIM12  
4  IWSIM12  


Verificando entregas de prácticas...
✗ P3 | ✓ P5 | AGAPITO DELGADO, SOFIA
✗ P3 | ✓ P5 | AGUILAR DESIAR, LLOYD DAREN
✗ P3 | ✓ P5 | AGUIRRE HERVIAS, JAVIER
✗ P3 | ✓ P5 | ALBRIZIO, MATEO
✗ P3 | ✗ P5 | ALONSO FERNANDEZ, NICOLAS
✗ P3 | ✓ P5 | ALVAREZ AREVALO, MIGUEL
✗ P3 | ✓ P5 | APUNTE SIERRA, AARON ALEJANDRO
✗ P3 | ✗ P5 | ARTACHO BORDINO, JORGE
✗ P3 | ✓ P5 | AUSIN MORENO, MARCOS
✗ P3 | ✓ P5 | AYALA MAYA, JULIO
✗ P3 | ✓ P5 | AYD

In [26]:
#display(df_con_practicas)

In [27]:
df_con_practicas.to_csv("../data/practicas_3_5")

In [28]:
# Seleccionar un grupo en concreto
columnas_importantes = ['Nombre', 'Apellido(s)', 'Presentada_5', 'Comentario_5']
df_con_practicas[df_con_practicas['Grupos'] == 'CITIT11'][columnas_importantes].to_csv('../data/alumnos_CITIT11.csv', index=False)

In [29]:
def renombrar_archivos_en_lotes(ruta="../data/raw/"):
	"""
	Renombra todos los archivos en la carpeta dada como 'lote_1', 'lote_2', etc.
	Conserva la extensión original de cada archivo.
	"""
	archivos = sorted([f for f in os.listdir(ruta) if os.path.isfile(os.path.join(ruta, f))])
	for idx, nombre_original in enumerate(archivos, start=1):
		extension = os.path.splitext(nombre_original)[1]
		nuevo_nombre = f"lote_{idx}{extension}"
		ruta_origen = os.path.join(ruta, nombre_original)
		ruta_destino = os.path.join(ruta, nuevo_nombre)
		os.rename(ruta_origen, ruta_destino)
	print(f"Renombrados {len(archivos)} archivos en '{ruta}'.")
#renombrar_archivos_en_lotes()

In [30]:
import os
from pathlib import Path
from pypdf import PdfReader, PdfWriter

def crear_carpeta_examenes(base_dir="../data", nombre_base="examenes"):
    """
    Crea una carpeta nueva para los exámenes. Si ya existe, añade un sufijo numérico.
    """
    base_path = Path(base_dir)
    carpeta = base_path / nombre_base
    contador = 1
    while carpeta.exists():
        carpeta = base_path / f"{nombre_base}_{contador}"
        contador += 1
    carpeta.mkdir(parents=True)
    return carpeta

def dividir_pdf_en_examenes(pdf_path, carpeta_destino, nombre_base="examen"):
    """
    Divide un PDF en archivos de 2 páginas cada uno y los guarda en la carpeta destino.
    """
    reader = PdfReader(pdf_path)
    num_paginas = len(reader.pages)
    examen_idx = 1
    for i in range(0, num_paginas, 2):
        writer = PdfWriter()
        writer.add_page(reader.pages[i])
        if i+1 < num_paginas:
            writer.add_page(reader.pages[i+1])
        nombre_examen = f"{nombre_base}_{examen_idx}.pdf"
        ruta_examen = carpeta_destino / nombre_examen
        with open(ruta_examen, "wb") as f_out:
            writer.write(f_out)
        examen_idx += 1

def procesar_lotes_y_generar_examenes(ruta_lotes="../data/raw/", base_dir="../data", nombre_carpeta="examenes"):
    """
    Busca todos los archivos PDF en la carpeta de lotes, los divide de 2 en 2 páginas y los guarda en una carpeta nueva.
    """
    carpeta_destino = crear_carpeta_examenes(base_dir, nombre_carpeta)
    archivos_lote = sorted([f for f in os.listdir(ruta_lotes) if f.lower().endswith(".pdf")])
    examen_global_idx = 1
    for archivo in archivos_lote:
        ruta_pdf = Path(ruta_lotes) / archivo
        reader = PdfReader(ruta_pdf)
        num_paginas = len(reader.pages)
        for i in range(0, num_paginas, 2):
            writer = PdfWriter()
            writer.add_page(reader.pages[i])
            if i+1 < num_paginas:
                writer.add_page(reader.pages[i+1])
            nombre_examen = f"examen_{examen_global_idx}.pdf"
            ruta_examen = carpeta_destino / nombre_examen
            with open(ruta_examen, "wb") as f_out:
                writer.write(f_out)
            examen_global_idx += 1
    print(f"Exámenes generados en: {carpeta_destino}")

# Ejemplo de uso:
#procesar_lotes_y_generar_examenes()

In [31]:
import os
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader, PdfWriter
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Dropdown
from IPython.display import display, clear_output

class JupyterPDFReviewer:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.reader = PdfReader(pdf_path)
        self.total_pages = len(self.reader.pages)
        self.current_index = 0
        self.images = {}
        self.output_dir = Path("../data/saved")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.out = Output()
        self.status = Label(value="")  # Estado visual
        self._setup_widgets()
        self._show_pages()

    def _setup_widgets(self):
        self.btn_save1 = Button(description='Save Page 1', layout=Layout(width='120px'))
        self.btn_save12 = Button(description='Save Pages 1&2', layout=Layout(width='120px'))
        self.btn_save123 = Button(description='Save Pages 1-3', layout=Layout(width='120px'))
        self.btn_next = Button(description='Next (Skip 1)', layout=Layout(width='120px'))
        self.btn_prev = Button(description='Previous', layout=Layout(width='120px'))

        self.btn_save1.on_click(lambda x: self._save_pages([0]))
        self.btn_save12.on_click(lambda x: self._save_pages([0, 1]))
        self.btn_save123.on_click(lambda x: self._save_pages([0, 1, 2]))
        self.btn_next.on_click(lambda x: self._next_page())
        self.btn_prev.on_click(lambda x: self._prev_page())

        display(VBox([
            HBox([self.btn_prev, self.btn_save1, self.btn_save12, self.btn_save123, self.btn_next]),
            self.status,
            self.out
        ]))

    def _get_page_image(self, idx):
        if idx not in self.images and idx < self.total_pages:
            self.status.value = f"Cargando página {idx+1}..."
            try:
                img = convert_from_path(
                    self.pdf_path,
                    first_page=idx + 1,
                    last_page=idx + 1,
                    dpi=50,
                    fmt='jpeg',
                    thread_count=1
                )[0]
                self.images[idx] = img
            except Exception as e:
                self.status.value = f"Error cargando página {idx+1}"
                print(f"Error converting page {idx+1}: {e}")
                return None
        self.status.value = ""
        return self.images.get(idx)

    def _show_pages(self):
        with self.out:
            clear_output(wait=True)
            fig, axes = plt.subplots(1, 3, figsize=(15, 8))
            for i in range(3):
                page_idx = self.current_index + i
                axes[i].axis('off')
                if page_idx < self.total_pages:
                    img = self._get_page_image(page_idx)
                    if img is not None:
                        axes[i].imshow(img)
                        axes[i].set_title(f"Page {page_idx+1}")
                    else:
                        axes[i].set_title(f"Page {page_idx+1} (error)")
                else:
                    axes[i].set_title("No Page")
            plt.show()

    def _next_page(self):
        if self.current_index + 1 < self.total_pages:
            self.current_index += 1
            self._show_pages()

    def _prev_page(self):
        if self.current_index >= 1:
            self.current_index -= 1
            self._show_pages()

    def _save_pages(self, rel_indices):
        abs_indices = [self.current_index + i for i in rel_indices if self.current_index + i < self.total_pages]
        if not abs_indices:
            self.status.value = "No valid pages to save"
            return

        base_name = Path(self.pdf_path).stem  # Ejemplo: 'lote_1'
        lote = base_name
        examen_n = abs_indices[0] + 1  # Primer índice de página + 1
        output_path = self.output_dir / f"{lote}_examen_{examen_n}.pdf"

        writer = PdfWriter()
        for idx in abs_indices:
            writer.add_page(self.reader.pages[idx])
        with open(output_path, "wb") as f:
            writer.write(f)
        self.status.value = f"Guardado: {output_path.name}"

        # Avanzar tantas páginas como se han guardado
        avance = len(abs_indices)
        if self.current_index + avance < self.total_pages:
            self.current_index += avance
            self._show_pages()

def revisar_todos_los_lotes(ruta_lotes="../data/raw/"):
    archivos = sorted([f for f in os.listdir(ruta_lotes) if f.lower().endswith(".pdf")])
    if not archivos:
        print("No se encontraron lotes PDF en la carpeta.")
        return
    dropdown = Dropdown(options=archivos, description='Lote:', layout=Layout(width='50%'))
    out = Output()

    def on_select(change):
        with out:
            clear_output(wait=True)
            print(f"Revisando: {dropdown.value}")
            JupyterPDFReviewer(os.path.join(ruta_lotes, dropdown.value))

    dropdown.observe(on_select, names='value')
    display(VBox([dropdown, out]))
    # Mostrar el primero por defecto
    on_select({'new': archivos[0]})

# Ejemplo de uso:
# revisar_todos_los_lotes("../data/raw/")

In [32]:
#revisar_todos_los_lotes("../data/raw/") # --> solo cuando necesites procesar los lotes

In [33]:
import os
import json
import base64
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
import pandas as pd
import shutil
import io
import requests

def procesar_examenes_completo(carpeta_examenes="../data/saved/", output_dir="../data/examenes_procesados/"):
    """
    Procesa todos los exámenes de la carpeta, extrae información con OpenAI,
    los organiza por grupo y práctica, y genera un DataFrame de seguimiento.
    """
    # Crear directorio de salida
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Lista para almacenar información de exámenes procesados
    examenes_info = []
    
    # Obtener todos los archivos PDF
    pdf_files = list(Path(carpeta_examenes).glob("*.pdf"))
    
    print(f"Procesando {len(pdf_files)} exámenes...")
    
    # Variables necesarias para la API
    api_key = os.environ["OPENAI_API_KEY"]
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    for pdf_file in pdf_files:
        print(f"\nProcesando: {pdf_file.name}")
        
        try:
            # Procesar cada página del PDF
            reader = PdfReader(pdf_file)
            num_pages = len(reader.pages)
            
            for page_num in range(min(2, num_pages)):  # Solo primeras 2 páginas
                # Convertir página a imagen
                images = convert_from_path(
                    pdf_file, 
                    first_page=page_num + 1, 
                    last_page=page_num + 1, 
                    dpi=150,
                    fmt='jpeg'
                )
                
                if not images:
                    continue
                    
                # Convertir imagen a base64
                buffered = io.BytesIO()
                images[0].save(buffered, format="JPEG")
                base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
                
                # Llamar a OpenAI
                info = extraer_info_con_openai(base64_image, headers)
                
                if info and info.get('Apellidos') and info.get('Nombre'):
                    info['archivo_original'] = pdf_file.name
                    info['pagina'] = page_num + 1
                    
                    # Organizar archivo por grupo y práctica
                    mover_archivo_organizado(pdf_file, info, output_path)
                    
                    examenes_info.append(info)
                    practica = info.get('practica_detectada', 'desconocida')
                    print(f"✓ Extraído: {info['Nombre']} {info['Apellidos']} - Grupo: {info['Grupo']} - Práctica: {practica}")
                    break  # Si encontramos info en una página, no procesar más páginas
                    
        except Exception as e:
            print(f"✗ Error procesando {pdf_file.name}: {e}")
            continue
    
    # Crear DataFrame de seguimiento
    df_examenes = crear_dataframe_examenes(examenes_info)
    
    # Guardar DataFrame
    df_examenes.to_csv(output_path / "seguimiento_examenes.csv", index=False)
    
    print(f"\n📊 Procesamiento completado:")
    print(f"- Exámenes procesados: {len(examenes_info)}")
    print(f"- Archivo de seguimiento guardado en: {output_path / 'seguimiento_examenes.csv'}")
    
    return df_examenes

def extraer_info_con_openai(base64_image, headers):
    """Extrae información del examen usando OpenAI"""
    try:
        prompt_completo = f"""Extract the last name (Apellidos in Spanish), the first name (Nombre in Spanish) and the group (Grupo in Spanish)
         from the top of the image. You will find them handwritten after the labels `Apellidos`,  `Nombre` and `Grupo` respectively. 
         The fields of your JSON output will have those exact same label names.
         Also determine if this is a "Práctica de Listas" (practice 3) or "Práctica de Grafos" (practice 5) based on the content.
         Add a field called "practica_detectada" with value 3 for listas or 5 for grafos.
         Here is a list of expected students and their groups as a reference: 
         {guia_texto}
         """
        
        payload = {
            "model": "gpt-4o",
            "response_format": {"type": "json_object"},
            "messages": [
                {
                    "role": "system", 
                    "content": "You are a helpful assistant designed to see an exam and output JSON with the extracted information. The default group in case of empty string is extraviado",   
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_completo
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        
        response = requests.post(
            "https://api.openai.com/v1/chat/completions", 
            headers=headers, 
            json=payload
        )
        
        if response.status_code == 200:
            info_str = response.json()['choices'][0]['message']['content']
            info = json.loads(info_str)
            
            # Si el grupo está vacío, marcarlo como "extraviado"
            if not info.get("Grupo"):
                info["Grupo"] = "extraviado"
                
            return info
        else:
            print(f"Error en API OpenAI: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error extrayendo información: {e}")
        return None

def mover_archivo_organizado(archivo_original, info, output_path):
    """Organiza el archivo en carpetas por GRUPO y luego por PRÁCTICA"""
    try:
        practica = info.get('practica_detectada', 'desconocida')
        grupo = info.get('Grupo', 'extraviado')
        
        # Crear estructura de carpetas: GRUPO -> PRÁCTICA
        carpeta_grupo = output_path / grupo
        carpeta_practica = carpeta_grupo / f"Practica_{practica}"
        carpeta_practica.mkdir(parents=True, exist_ok=True)
        
        # Generar nombre de archivo
        apellidos = info.get('Apellidos', 'SinApellidos').replace(' ', '_')
        nombre = info.get('Nombre', 'SinNombre').replace(' ', '_')
        
        nombre_base = f"{apellidos}_{nombre}"
        extension = archivo_original.suffix
        nuevo_archivo = carpeta_practica / f"{nombre_base}{extension}"
        
        # Si ya existe, añadir sufijo numérico
        contador = 2
        while nuevo_archivo.exists():
            nuevo_archivo = carpeta_practica / f"{nombre_base}_{contador}{extension}"
            contador += 1
        
        # Copiar archivo
        shutil.copy2(archivo_original, nuevo_archivo)
        print(f"  → Guardado en: {carpeta_grupo.name}/{carpeta_practica.name}/{nuevo_archivo.name}")
        
    except Exception as e:
        print(f"Error organizando archivo {archivo_original.name}: {e}")

def crear_dataframe_examenes(examenes_info):
    """Crea DataFrame de seguimiento de exámenes"""
    # Crear DataFrame base con todos los estudiantes
    df_base = df[['Nombre', 'Apellido(s)', 'Grupos']].copy()
    
    # Inicializar columnas de exámenes
    df_base['Examen_3'] = 0
    df_base['Comentario_Examen_3'] = 'PNP'  # PNP = No Presentado
    df_base['Examen_5'] = 0
    df_base['Comentario_Examen_5'] = 'PNP'  # PNP = No Presentado
    
    # Procesar información de exámenes
    for examen in examenes_info:
        practica = examen.get('practica_detectada')
        if practica in [3, 5, '3', '5']:
            practica = str(practica)
            
            # Buscar estudiante en el DataFrame
            apellidos = examen.get('Apellidos', '').upper().strip()
            nombre = examen.get('Nombre', '').upper().strip()
            
            # Buscar coincidencia (más flexible)
            mask = df_base['Apellido(s)'].str.upper().str.contains(apellidos[:5] if len(apellidos) > 5 else apellidos, na=False, regex=False) & \
                   df_base['Nombre'].str.upper().str.contains(nombre[:5] if len(nombre) > 5 else nombre, na=False, regex=False)
            
            if mask.any():
                df_base.loc[mask, f'Examen_{practica}'] = 1
                df_base.loc[mask, f'Comentario_Examen_{practica}'] = ''  # Limpiar PNP
                print(f"  → Marcado como presentado: {nombre} {apellidos} - Práctica {practica}")
    
    return df_base



In [34]:
import os
import json
import base64
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
import pandas as pd
import shutil
import io
import requests
from rapidfuzz import fuzz
import re
import unicodedata

def limpiar_texto(texto):
    """Elimina acentos, símbolos y deja solo letras/números/espacios en mayúsculas"""
    if not texto:
        return ""
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')
    texto = texto.upper()
    texto = re.sub(r'[^A-Z0-9\s]', '', texto)
    texto = texto.strip()
    return texto

def buscar_grupo_flexible(nombre, apellidos, df, texto_ocr=None, umbral=90):
    """
    Busca el grupo del alumno en el DataFrame por nombre y apellidos usando fuzzy.
    Si no encuentra por el umbral, devuelve el grupo más parecido aunque el score sea bajo (mínimo 20%).
    """
    nombre = limpiar_texto(nombre)
    apellidos = limpiar_texto(apellidos)
    mejor_score = -1
    mejor_grupo = None
    mejor_nombre = ""
    mejor_apellidos = ""
    for _, row in df.iterrows():
        nombre_df = limpiar_texto(str(row["Nombre"]))
        apellidos_df = limpiar_texto(str(row["Apellido(s)"]))
        score_nombre = fuzz.ratio(nombre, nombre_df)
        score_apellidos = fuzz.ratio(apellidos, apellidos_df)
        score = (score_nombre + score_apellidos) / 2
        if score > mejor_score:
            mejor_score = score
            mejor_grupo = row["Grupos"]
            mejor_nombre = row["Nombre"]
            mejor_apellidos = row["Apellido(s)"]
    # Si supera el umbral, devuelve el grupo y nombre exactos
    if mejor_score >= umbral:
        return mejor_grupo, mejor_nombre, mejor_apellidos
    # Si no supera el umbral pero hay algún match > 20%, devuelve el más parecido
    if mejor_score >= 20:
        return mejor_grupo, mejor_nombre, mejor_apellidos
    # Si no hay nada ni con 20%, busca patrón OCR
    if texto_ocr:
        texto_ocr = texto_ocr.upper()
        patrones = [
            r"CITI[TM][1][12]", r"IWSI[TM][1][12]", r"IWSIT[1][12]", r"CITIT[1][12]", r"IWSIM[1][12]"
        ]
        for patron in patrones:
            match = re.search(patron, texto_ocr)
            if match:
                return match.group(0), "", ""
    return "extraviado", "", ""

def extraer_info_con_openai(base64_image, headers, df):
    try:
        prompt_completo = (
            "Extract the last name (Apellidos in Spanish), "
            "the first name (Nombre in Spanish) from the top of the image. "
            "Extract also the group (Grupo in Spanish) from the top of the image. "
            "You will find them handwritten after the labels `Apellidos` and `Nombre` and `Group` respectively. "
            "The fields of your JSON output will have those exact same label names. "
            "Also determine if this is a \"Práctica de Listas\" (practice 3) or \"Práctica de Grafos\" (practice 5) based on the content. "
            "Add a field called \"practica_detectada\" with value 3 for listas or 5 for grafos. "
            "If you see a group label like CITIM11, CITIM12, IWSIM11, IWSIM12, CITIT11, IWSIT11, IWSIT12, include it as the field 'Grupo'."
        )
        payload = {
            "model": "gpt-4o",
            "response_format": {"type": "json_object"},
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant designed to see an exam and output JSON with the extracted information."
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_completo},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                    ]
                }
            ],
            "max_tokens": 300
        }
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload,
                timeout=10
            )
        except requests.exceptions.Timeout:
            return None, "Timeout: La petición a OpenAI tardó más de 10 segundos."
        except requests.exceptions.RequestException as e:
            return None, f"Error de conexión con OpenAI: {e}"

        if response.status_code == 200:
            info_str = response.json()['choices'][0]['message']['content']
            try:
                info = json.loads(info_str)
                nombre_ocr = info.get("Nombre", "")
                apellidos_ocr = info.get("Apellidos", "")
                grupo_ocr = info.get("Grupo", "")
                grupo, nombre, apellidos = buscar_grupo_flexible(nombre_ocr, apellidos_ocr, df, texto_ocr=grupo_ocr)
                info["Grupo"] = grupo
                # Si se encontró en el excel, usa los nombres del excel (más limpios)
                if grupo != "extraviado" and nombre and apellidos:
                    info["Nombre"] = nombre
                    info["Apellidos"] = apellidos
                else:
                    # Si no, limpia los nombres extraídos por OCR
                    info["Nombre"] = limpiar_texto(nombre_ocr)
                    info["Apellidos"] = limpiar_texto(apellidos_ocr)
                if grupo == "extraviado" and grupo_ocr:
                    info["Grupo_detectado"] = grupo_ocr
                return info, None
            except json.JSONDecodeError as e:
                return None, f"Error parseando JSON: {e}\nRespuesta recibida: {info_str}"
        else:
            return None, f"Error en API OpenAI: {response.status_code} - {response.text}"
    except Exception as e:
        return None, f"Error extrayendo información: {e}"

def extraer_solo_practica(base64_image, headers):
    try:
        prompt_practica = (
            "Look at this exam image and determine if this is a \"Práctica de Listas\" (practice 3) "
            "or \"Práctica de Grafos\" (practice 5) based on the content. "
            "Return JSON with fields: \"Apellidos\": \"\", \"Nombre\": \"\", \"Grupo\": \"extraviado\", \"practica_detectada\": 3 or 5"
        )
        payload = {
            "model": "gpt-4o",
            "response_format": {"type": "json_object"},
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant that identifies exam types and outputs JSON.",
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_practica},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                    ]
                }
            ],
            "max_tokens": 150
        }
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload,
                timeout=10
            )
        except requests.exceptions.Timeout:
            return None, "Timeout: La petición a OpenAI tardó más de 10 segundos (solo práctica)."
        except requests.exceptions.RequestException as e:
            return None, f"Error de conexión con OpenAI (solo práctica): {e}"

        if response.status_code == 200:
            info_str = response.json()['choices'][0]['message']['content']
            try:
                info = json.loads(info_str)
                return info, None
            except json.JSONDecodeError:
                return None, f"Error parseando JSON (solo práctica): {info_str}"
        else:
            return None, f"Error en API OpenAI (solo práctica): {response.status_code} - {response.text}"
    except Exception as e:
        return None, f"Error en retry de práctica: {e}"

def extraer_numero_lote(nombre):
    match = re.search(r"lote_(\d+)", nombre)
    return int(match.group(1)) if match else 0

def mover_archivo_organizado(archivo_original, info, output_path):
    try:
        practica = info.get('practica_detectada', 'desconocida')
        grupo = info.get('Grupo', 'extraviado')
        carpeta_grupo = output_path / grupo
        carpeta_practica = carpeta_grupo / f"Practica_{practica}"
        carpeta_practica.mkdir(parents=True, exist_ok=True)
        apellidos = info.get('Apellidos', '').strip()
        nombre = info.get('Nombre', '').strip()
        if grupo == "extraviado" and not apellidos and not nombre:
            nombre_base = Path(info.get('archivo_original', archivo_original.name)).stem
        else:
            apellidos = apellidos if apellidos else 'SinApellidos'
            nombre = nombre if nombre else 'SinNombre'
            nombre_base = f"{apellidos.replace(' ', '_')}_{nombre.replace(' ', '_')}"
            if grupo == "extraviado" and info.get('Grupo_detectado'):
                nombre_base += f"_{info['Grupo_detectado']}"
        extension = archivo_original.suffix
        nuevo_archivo = carpeta_practica / f"{nombre_base}{extension}"
        contador = 2
        while nuevo_archivo.exists():
            nuevo_archivo = carpeta_practica / f"{nombre_base}_{contador}{extension}"
            contador += 1
        shutil.copy2(archivo_original, nuevo_archivo)
        print(f"  → Guardado en: {carpeta_grupo.name}/{carpeta_practica.name}/{nuevo_archivo.name}")
    except Exception as e:
        print(f"Error organizando archivo {archivo_original.name}: {e}")

def crear_dataframe_examenes(examenes_info, df):
    df_base = df[['Nombre', 'Apellido(s)', 'Grupos']].copy()
    df_base['Examen_3'] = 0
    df_base['Comentario_Examen_3'] = 'PNP'
    df_base['Examen_5'] = 0
    df_base['Comentario_Examen_5'] = 'PNP'
    for examen in examenes_info:
        practica = examen.get('practica_detectada')
        if practica in [3, 5, '3', '5']:
            practica = str(practica)
            apellidos = examen.get('Apellidos', '').upper().strip()
            nombre = examen.get('Nombre', '').upper().strip()
            if apellidos and nombre:
                mask = df_base['Apellido(s)'].str.upper().str.strip() == apellidos
                mask &= df_base['Nombre'].str.upper().str.strip() == nombre
                if mask.any():
                    df_base.loc[mask, f'Examen_{practica}'] = 1
                    df_base.loc[mask, f'Comentario_Examen_{practica}'] = ''
                    print(f"  → Marcado como presentado: {nombre} {apellidos} - Práctica {practica}")
    return df_base

def procesar_examenes_completo(carpeta_examenes="../data/saved/", output_dir="../data/examenes_procesados/"):
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    extraviados_path = output_path / "extraviados"
    extraviados_path.mkdir(parents=True, exist_ok=True)
    examenes_info = []
    errores_openai = []
    # Ordenar por número de lote, no alfabéticamente
    pdf_files = sorted(Path(carpeta_examenes).glob("*.pdf"), key=lambda x: extraer_numero_lote(x.name))
    print(f"Procesando {len(pdf_files)} exámenes...")
    api_key = os.environ["OPENAI_API_KEY"]
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    # Cargar DataFrame de alumnos
    students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
    dfs = [ pd.read_csv(filename) for filename in students_info ]
    df = pd.concat(dfs, ignore_index=True)
    df["Nombre"] = df["Nombre"].str.strip().str.upper()
    df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
    for pdf_file in pdf_files:
        print(f"\nProcesando: {pdf_file.name}")
        try:
            reader = PdfReader(pdf_file)
            num_pages = len(reader.pages)
            info_extraida = None
            error_detalle = None
            for page_num in range(min(2, num_pages)):
                images = convert_from_path(
                    pdf_file,
                    first_page=page_num + 1,
                    last_page=page_num + 1,
                    dpi=150,
                    fmt='jpeg'
                )
                if not images:
                    continue
                buffered = io.BytesIO()
                images[0].save(buffered, format="JPEG")
                base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
                info, error = extraer_info_con_openai(base64_image, headers, df)
                if info and info.get('Apellidos') and info.get('Nombre'):
                    info['archivo_original'] = pdf_file.name
                    info['pagina'] = page_num + 1
                    info_extraida = info
                    break
                elif info and not info.get('Apellidos'):
                    info_retry, error_retry = extraer_solo_practica(base64_image, headers)
                    if info_retry:
                        info_extraida = info_retry
                        info_extraida['archivo_original'] = pdf_file.name
                        info_extraida['pagina'] = page_num + 1
                        break
                    elif error_retry:
                        error_detalle = error_retry
                elif error:
                    error_detalle = error
            if info_extraida:
                mover_archivo_organizado(pdf_file, info_extraida, output_path)
                examenes_info.append(info_extraida)
                practica = info_extraida.get('practica_detectada', 'desconocida')
                print(f"✓ Extraído: {info_extraida.get('Nombre', 'Sin nombre')} {info_extraida.get('Apellidos', 'Sin apellidos')} - Grupo: {info_extraida.get('Grupo', 'extraviado')} - Práctica: {practica}")
            else:
                shutil.copy2(pdf_file, extraviados_path / pdf_file.name)
                print(f"✗ No se pudo extraer información - Copiado a extraviados: {pdf_file.name}")
                errores_openai.append({
                    "archivo": pdf_file.name,
                    "error": error_detalle or "No se pudo extraer información ni con retry",
                })
        except Exception as e:
            print(f"✗ Error procesando {pdf_file.name}: {e}")
            try:
                shutil.copy2(pdf_file, extraviados_path / pdf_file.name)
                print(f"  → Copiado a extraviados por error")
            except:
                pass
            errores_openai.append({
                "archivo": pdf_file.name,
                "error": str(e),
            })
            continue
    df_examenes = crear_dataframe_examenes(examenes_info, df)
    df_examenes.to_csv(output_path / "seguimiento_examenes.csv", index=False)
    print(f"\n📊 Procesamiento completado:")
    print(f"- Exámenes procesados: {len(examenes_info)}")
    print(f"- Archivos en extraviados: {len(list(extraviados_path.glob('*.pdf')))}")
    print(f"- Archivo de seguimiento guardado en: {output_path / 'seguimiento_examenes.csv'}")
    # Crear DataFrame de errores y mostrarlo
    if errores_openai:
        df_errores = pd.DataFrame(errores_openai)
        df_errores.to_csv(output_path / "errores_openai.csv", index=False)
        print(f"\n❗ Casos fallidos por OpenAI guardados en: {output_path / 'errores_openai.csv'}")
        print(df_errores)
    else:
        print("\n✅ No hubo errores de extracción con OpenAI.")
    return df_examenes

In [35]:
# Ejecutar el procesamiento
#df_examenes_procesados = procesar_examenes_completo()
print("\n📊 Resumen de exámenes procesados por grupo:")
#print(df_examenes_procesados.groupby('Grupos')[['Examen_3', 'Examen_5']].sum())


📊 Resumen de exámenes procesados por grupo:


In [36]:
import os
import shutil
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Text, Dropdown, Checkbox
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import re
import matplotlib.pyplot as plt

class JupyterExamReviewer:
    def __init__(self, examenes_dir="../data/examenes_procesados/"):
        self.examenes_dir = Path(examenes_dir)
        self.current_index = 0
        self.exam_files = []
        self.changes_pending = {}  # Diccionario para almacenar cambios pendientes
        self.out = Output()
        self.status = Label(value="Cargando exámenes...")
        
        # Cargar todos los archivos PDF
        self._load_exam_files()
        
        if not self.exam_files:
            self.status.value = "No se encontraron exámenes para revisar"
            return
            
        # Configurar widgets
        self._setup_widgets()
        self._show_current_exam()

    def _load_exam_files(self):
        """Carga todos los archivos PDF de todas las carpetas de grupos y prácticas"""
        print("🔍 Escaneando carpetas de exámenes...")
        
        for grupo_dir in self.examenes_dir.iterdir():
            if grupo_dir.is_dir() and grupo_dir.name not in ["extraviados", "problemático"]:
                for practica_dir in grupo_dir.iterdir():
                    if practica_dir.is_dir() and practica_dir.name.startswith("Practica_"):
                        for pdf_file in practica_dir.glob("*.pdf"):
                            self.exam_files.append({
                                'file_path': pdf_file,
                                'grupo_actual': grupo_dir.name,
                                'practica': practica_dir.name,
                                'nombre_archivo': pdf_file.stem
                            })
        
        # Ordenar por grupo y luego por nombre
        self.exam_files.sort(key=lambda x: (x['grupo_actual'], x['nombre_archivo']))
        print(f"📁 Total de exámenes encontrados: {len(self.exam_files)}")

    def _setup_widgets(self):
        """Configura todos los widgets de la interfaz"""
        # Botones de navegación
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='120px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='120px'))
        
        # Campos de edición
        self.txt_nombre = Text(description='Nombre:', layout=Layout(width='300px'))
        self.txt_apellidos = Text(description='Apellidos:', layout=Layout(width='300px'))
        
        # Dropdown para cambiar grupo
        grupos_disponibles = [d.name for d in self.examenes_dir.iterdir() 
                             if d.is_dir() and d.name not in ["extraviados", "problemático"]]
        grupos_disponibles.extend(["extraviados", "problemático"])  # Añadir opciones especiales
        
        self.dropdown_grupo = Dropdown(
            options=grupos_disponibles,
            description='Grupo:',
            layout=Layout(width='200px')
        )
        
        # Checkboxes para confirmar carpetas correctas
        self.check_nombre_ok = Checkbox(description='Nombre correcto', value=True)
        self.check_grupo_ok = Checkbox(description='Grupo correcto', value=True)
        
        # Botón para aplicar cambios
        self.btn_apply = Button(
            description='✓ Aplicar Cambios',
            button_style='success',
            layout=Layout(width='150px')
        )
        
        # Contador de progreso
        self.progress_label = Label(value="")
        
        # Conectar eventos
        self.btn_prev.on_click(lambda x: self._navigate(-1))
        self.btn_next.on_click(lambda x: self._navigate(1))
        self.btn_apply.on_click(lambda x: self._apply_changes())
        
        # Layout de la interfaz
        controls_row1 = HBox([
            self.btn_prev, 
            self.progress_label,
            self.btn_next
        ])
        
        edit_row = HBox([
            self.txt_nombre,
            self.txt_apellidos,
            self.dropdown_grupo
        ])
        
        check_row = HBox([
            self.check_nombre_ok,
            self.check_grupo_ok,
            self.btn_apply
        ])
        
        self.interface = VBox([
            controls_row1,
            edit_row,
            check_row,
            self.status,
            self.out
        ])
        
        display(self.interface)

    def _show_current_exam(self):
        """Muestra el examen actual"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        
        # Actualizar contador de progreso
        self.progress_label.value = f"Examen {self.current_index + 1} de {len(self.exam_files)}"
        
        # Actualizar campos con información actual
        nombre_completo = current_exam['nombre_archivo']
        partes = nombre_completo.replace('_', ' ').split()
        
        if len(partes) >= 2:
            self.txt_apellidos.value = ' '.join(partes[:-1])
            self.txt_nombre.value = partes[-1]
        else:
            self.txt_apellidos.value = nombre_completo
            self.txt_nombre.value = ""
            
        self.dropdown_grupo.value = current_exam['grupo_actual']
        
        # Resetear checkboxes
        self.check_nombre_ok.value = True
        self.check_grupo_ok.value = True
        
        # Mostrar información del archivo
        self.status.value = f"📁 {current_exam['grupo_actual']} / {current_exam['practica']} / {current_exam['nombre_archivo']}.pdf"
        
        # Mostrar imagen del examen
        self._show_exam_image(current_exam['file_path'])

    def _show_exam_image(self, pdf_path):
        """Muestra las páginas del PDF (hasta 2 páginas)"""
        with self.out:
            clear_output(wait=True)
            try:
                # Contar páginas primero
                reader = PdfReader(pdf_path)
                num_pages = len(reader.pages)
                
                # Si el PDF no tiene exactamente 2 páginas, ofrecer opción "problemático"
                if num_pages != 2:
                    print(f"⚠️ El PDF tiene {num_pages} página(s) en lugar de 2")
                    print("💡 Considera moverlo a la carpeta 'problemático'")
                    # Actualizar dropdown para mostrar "problemático" como opción sugerida
                    if "problemático" not in [opt for opt in self.dropdown_grupo.options]:
                        self.dropdown_grupo.value = "problemático"
                
                # Convertir hasta 2 páginas
                pages_to_show = min(2, num_pages)
                images = convert_from_path(
                    pdf_path,
                    first_page=1,
                    last_page=pages_to_show,
                    dpi=150,
                    fmt='jpeg'
                )
                
                if images:
                    # Configurar subplots según el número de imágenes
                    if len(images) == 1:
                        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
                        axes = [ax]
                    else:
                        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
                    
                    for i, image in enumerate(images):
                        # Recortar la parte superior (donde están los datos del estudiante)
                        if i == 0:  # Primera página - cabecera
                            cropped = image.crop((0, 0, image.width, min(600, image.height // 3)))
                            title = f"Página {i+1} - Cabecera"
                        else:  # Segunda página - completa pero más pequeña
                            cropped = image.crop((0, 0, image.width, image.height))
                            title = f"Página {i+1} - Completa"
                        
                        axes[i].imshow(cropped)
                        axes[i].axis('off')
                        axes[i].set_title(title)
                    
                    plt.suptitle(f"Examen: {pdf_path.name} ({num_pages} página(s))")
                    plt.tight_layout()
                    plt.show()
                else:
                    print("❌ No se pudo cargar la imagen del PDF")
                    
            except Exception as e:
                print(f"❌ Error cargando imagen: {e}")

    def _navigate(self, direction):
        """Navega al examen anterior o siguiente"""
        # Aplicar cambios pendientes si los hay
        if self._has_pending_changes():
            self._apply_changes()
        
        # Cambiar índice
        new_index = self.current_index + direction
        
        if 0 <= new_index < len(self.exam_files):
            self.current_index = new_index
            self._show_current_exam()
        elif new_index >= len(self.exam_files):
            self.status.value = "🎉 ¡Revisión completada! Has llegado al final."
        elif new_index < 0:
            self.status.value = "📍 Ya estás en el primer examen."

    def _has_pending_changes(self):
        """Verifica si hay cambios pendientes"""
        if not self.exam_files:
            return False
            
        current_exam = self.exam_files[self.current_index]
        
        # Verificar cambios en nombre/apellidos
        nombre_actual = self.txt_nombre.value.strip().upper()
        apellidos_actual = self.txt_apellidos.value.strip().upper()
        nuevo_nombre = f"{apellidos_actual}_{nombre_actual}".replace(' ', '_')
        
        nombre_archivo_actual = current_exam['nombre_archivo']
        
        # Verificar cambio de grupo
        grupo_cambio = self.dropdown_grupo.value != current_exam['grupo_actual']
        nombre_cambio = nuevo_nombre != nombre_archivo_actual
        
        return grupo_cambio or nombre_cambio or not self.check_nombre_ok.value or not self.check_grupo_ok.value

    def _apply_changes(self):
        """Aplica los cambios al archivo actual"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        old_path = current_exam['file_path']
        
        try:
            # Preparar nuevo nombre de archivo
            nombre_nuevo = self.txt_nombre.value.strip().replace(' ', '_')
            apellidos_nuevo = self.txt_apellidos.value.strip().replace(' ', '_')
            nuevo_nombre_archivo = f"{apellidos_nuevo}_{nombre_nuevo}"
            
            # Preparar nueva ubicación
            nuevo_grupo = self.dropdown_grupo.value
            practica_actual = current_exam['practica']
            
            nueva_carpeta = self.examenes_dir / nuevo_grupo
            if nuevo_grupo not in ["extraviados", "problemático"]:
                nueva_carpeta = nueva_carpeta / practica_actual
            
            # Crear carpeta si no existe
            nueva_carpeta.mkdir(parents=True, exist_ok=True)
            
            # Nuevo path completo
            nuevo_path = nueva_carpeta / f"{nuevo_nombre_archivo}.pdf"
            
            # Evitar conflictos de nombres
            contador = 2
            while nuevo_path.exists() and nuevo_path != old_path:
                nuevo_path = nueva_carpeta / f"{nuevo_nombre_archivo}_{contador}.pdf"
                contador += 1
            
            # Mover archivo si es necesario
            if nuevo_path != old_path:
                shutil.move(str(old_path), str(nuevo_path))
                
                # Actualizar información en la lista
                current_exam['file_path'] = nuevo_path
                current_exam['grupo_actual'] = nuevo_grupo
                current_exam['nombre_archivo'] = nuevo_path.stem
                
                self.status.value = f"✅ Movido a: {nuevo_grupo}/{nuevo_path.name}"
            else:
                self.status.value = "ℹ️ Sin cambios necesarios"
                
        except Exception as e:
            self.status.value = f"❌ Error aplicando cambios: {e}"

def iniciar_revision_examenes(examenes_dir="../data/examenes_procesados/"):
    """Inicia la interfaz de revisión de exámenes"""
    print("🚀 Iniciando revisor de exámenes...")
    reviewer = JupyterExamReviewer(examenes_dir)
    return reviewer


In [37]:
#reviewer = iniciar_revision_examenes()

In [38]:
import os
import shutil
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Text, Dropdown, Checkbox
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import re

import matplotlib.pyplot as plt

class JupyterExamReviewer:
    def __init__(self, examenes_dir="../data/examenes_procesados/"):
        self.examenes_dir = Path(examenes_dir)
        self.current_index = 0
        self.exam_files = []
        self.changes_pending = {}  # Diccionario para almacenar cambios pendientes
        self.out = Output()
        self.status = Label(value="Cargando exámenes...")
        
        # Cargar todos los archivos PDF
        self._load_exam_files()
        
        if not self.exam_files:
            self.status.value = "No se encontraron exámenes para revisar"
            return
            
        # Configurar widgets
        self._setup_widgets()
        self._show_current_exam()

    def _load_exam_files(self):
        """Carga todos los archivos PDF de todas las carpetas de grupos y prácticas"""
        print("🔍 Escaneando carpetas de exámenes...")
        
        for grupo_dir in self.examenes_dir.iterdir():
            if grupo_dir.is_dir() and grupo_dir.name not in ["extraviados", "problemático"]:
                for practica_dir in grupo_dir.iterdir():
                    if practica_dir.is_dir() and practica_dir.name.startswith("Practica_"):
                        for pdf_file in practica_dir.glob("*.pdf"):
                            self.exam_files.append({
                                'file_path': pdf_file,
                                'grupo_actual': grupo_dir.name,
                                'practica': practica_dir.name,
                                'nombre_archivo': pdf_file.stem
                            })
        
        # También cargar archivos de carpetas especiales
        for special_dir in ["extraviados", "problemático"]:
            special_path = self.examenes_dir / special_dir
            if special_path.exists():
                for pdf_file in special_path.glob("*.pdf"):
                    self.exam_files.append({
                        'file_path': pdf_file,
                        'grupo_actual': special_dir,
                        'practica': "",  # Sin práctica para carpetas especiales
                        'nombre_archivo': pdf_file.stem
                    })
        
        # Ordenar por grupo y luego por nombre
        self.exam_files.sort(key=lambda x: (x['grupo_actual'], x['nombre_archivo']))
        print(f"📁 Total de exámenes encontrados: {len(self.exam_files)}")

    def _setup_widgets(self):
        """Configura todos los widgets de la interfaz"""
        # Botones de navegación
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='120px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='120px'))
        
        # Campos de edición
        self.txt_nombre = Text(description='Nombre:', layout=Layout(width='300px'))
        self.txt_apellidos = Text(description='Apellidos:', layout=Layout(width='300px'))
        
        # Dropdown para cambiar grupo
        grupos_disponibles = [d.name for d in self.examenes_dir.iterdir() 
                             if d.is_dir() and d.name not in ["extraviados", "problemático"]]
        grupos_disponibles.extend(["extraviados", "problemático"])  # Añadir opciones especiales
        
        self.dropdown_grupo = Dropdown(
            options=grupos_disponibles,
            description='Grupo:',
            layout=Layout(width='200px')
        )
        
        # Dropdown para cambiar práctica
        self.dropdown_practica = Dropdown(
            options=['2', '3', '4', '5'],
            description='Práctica:',
            layout=Layout(width='150px')
        )
        
        # Checkboxes para confirmar carpetas correctas
        self.check_nombre_ok = Checkbox(description='Nombre correcto', value=True)
        self.check_grupo_ok = Checkbox(description='Grupo correcto', value=True)
        
        # Botón para aplicar cambios
        self.btn_apply = Button(
            description='✓ Aplicar Cambios',
            button_style='success',
            layout=Layout(width='150px')
        )
        
        # Contador de progreso
        self.progress_label = Label(value="")
        
        # Conectar eventos
        self.btn_prev.on_click(lambda x: self._navigate(-1))
        self.btn_next.on_click(lambda x: self._navigate(1))
        self.btn_apply.on_click(lambda x: self._apply_changes())
        self.dropdown_grupo.observe(self._on_grupo_change, names='value')
        
        # Layout de la interfaz
        controls_row1 = HBox([
            self.btn_prev, 
            self.progress_label,
            self.btn_next
        ])
        
        edit_row = HBox([
            self.txt_nombre,
            self.txt_apellidos,
            self.dropdown_grupo,
            self.dropdown_practica
        ])
        
        check_row = HBox([
            self.check_nombre_ok,
            self.check_grupo_ok,
            self.btn_apply
        ])
        
        self.interface = VBox([
            controls_row1,
            edit_row,
            check_row,
            self.status,
            self.out
        ])
        
        display(self.interface)

    def _on_grupo_change(self, change):
        """Maneja el cambio de grupo para habilitar/deshabilitar dropdown de práctica"""
        if change['new'] in ["extraviados", "problemático"]:
            self.dropdown_practica.disabled = True
        else:
            self.dropdown_practica.disabled = False

    def _show_current_exam(self):
        """Muestra el examen actual"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        
        # Actualizar contador de progreso
        self.progress_label.value = f"Examen {self.current_index + 1} de {len(self.exam_files)}"
        
        # Actualizar campos con información actual
        nombre_completo = current_exam['nombre_archivo']
        partes = nombre_completo.replace('_', ' ').split()
        
        if len(partes) >= 2:
            self.txt_apellidos.value = ' '.join(partes[:-1])
            self.txt_nombre.value = partes[-1]
        else:
            self.txt_apellidos.value = nombre_completo
            self.txt_nombre.value = ""
            
        self.dropdown_grupo.value = current_exam['grupo_actual']
        
        # Extraer número de práctica actual
        if current_exam['practica']:
            practica_num = current_exam['practica'].replace('Practica_', '')
            if practica_num in ['2', '3', '4', '5']:
                self.dropdown_practica.value = practica_num
            else:
                self.dropdown_practica.value = '3'  # valor por defecto
        else:
            self.dropdown_practica.value = '3'  # valor por defecto para carpetas especiales
        
        # Habilitar/deshabilitar dropdown de práctica
        if current_exam['grupo_actual'] in ["extraviados", "problemático"]:
            self.dropdown_practica.disabled = True
        else:
            self.dropdown_practica.disabled = False
        
        # Resetear checkboxes
        self.check_nombre_ok.value = True
        self.check_grupo_ok.value = True
        
        # Mostrar información del archivo
        if current_exam['practica']:
            self.status.value = f"📁 {current_exam['grupo_actual']} / {current_exam['practica']} / {current_exam['nombre_archivo']}.pdf"
        else:
            self.status.value = f"📁 {current_exam['grupo_actual']} / {current_exam['nombre_archivo']}.pdf"
        
        # Mostrar imagen del examen
        self._show_exam_image(current_exam['file_path'])

    def _show_exam_image(self, pdf_path):
        """Muestra las páginas del PDF (hasta 2 páginas)"""
        with self.out:
            clear_output(wait=True)
            try:
                # Contar páginas primero
                reader = PdfReader(pdf_path)
                num_pages = len(reader.pages)
                
                # Si el PDF no tiene exactamente 2 páginas, ofrecer opción "problemático"
                if num_pages != 2:
                    print(f"⚠️ El PDF tiene {num_pages} página(s) en lugar de 2")
                    print("💡 Considera moverlo a la carpeta 'problemático'")
                
                # Convertir hasta 2 páginas
                pages_to_show = min(2, num_pages)
                images = convert_from_path(
                    pdf_path,
                    first_page=1,
                    last_page=pages_to_show,
                    dpi=150,
                    fmt='jpeg'
                )
                
                if images:
                    # Configurar subplots según el número de imágenes
                    if len(images) == 1:
                        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
                        axes = [ax]
                    else:
                        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
                    
                    for i, image in enumerate(images):
                        # Recortar la parte superior (donde están los datos del estudiante)
                        if i == 0:  # Primera página - cabecera
                            cropped = image.crop((0, 0, image.width, min(600, image.height // 3)))
                            title = f"Página {i+1} - Cabecera"
                        else:  # Segunda página - completa pero más pequeña
                            cropped = image.crop((0, 0, image.width, image.height))
                            title = f"Página {i+1} - Completa"
                        
                        axes[i].imshow(cropped)
                        axes[i].axis('off')
                        axes[i].set_title(title)
                    
                    plt.suptitle(f"Examen: {pdf_path.name} ({num_pages} página(s))")
                    plt.tight_layout()
                    plt.show()
                else:
                    print("❌ No se pudo cargar la imagen del PDF")
                    
            except Exception as e:
                print(f"❌ Error cargando imagen: {e}")

    def _navigate(self, direction):
        """Navega al examen anterior o siguiente"""
        # Aplicar cambios pendientes si los hay
        if self._has_pending_changes():
            self._apply_changes()
        
        # Cambiar índice
        new_index = self.current_index + direction
        
        if 0 <= new_index < len(self.exam_files):
            self.current_index = new_index
            self._show_current_exam()
        elif new_index >= len(self.exam_files):
            self.status.value = "🎉 ¡Revisión completada! Has llegado al final."
        elif new_index < 0:
            self.status.value = "📍 Ya estás en el primer examen."

    def _has_pending_changes(self):
        """Verifica si hay cambios pendientes"""
        if not self.exam_files:
            return False
            
        current_exam = self.exam_files[self.current_index]
        
        # Verificar cambios en nombre/apellidos
        nombre_actual = self.txt_nombre.value.strip().upper()
        apellidos_actual = self.txt_apellidos.value.strip().upper()
        nuevo_nombre = f"{apellidos_actual}_{nombre_actual}".replace(' ', '_')
        
        nombre_archivo_actual = current_exam['nombre_archivo']
        
        # Verificar cambio de grupo
        grupo_cambio = self.dropdown_grupo.value != current_exam['grupo_actual']
        
        # Verificar cambio de práctica
        practica_actual = current_exam['practica'].replace('Practica_', '') if current_exam['practica'] else ''
        practica_cambio = self.dropdown_practica.value != practica_actual and not self.dropdown_practica.disabled
        
        nombre_cambio = nuevo_nombre != nombre_archivo_actual
        
        return grupo_cambio or practica_cambio or nombre_cambio or not self.check_nombre_ok.value or not self.check_grupo_ok.value

    def _apply_changes(self):
        """Aplica los cambios al archivo actual"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        old_path = current_exam['file_path']
        
        try:
            # Preparar nuevo nombre de archivo
            nombre_nuevo = self.txt_nombre.value.strip().replace(' ', '_')
            apellidos_nuevo = self.txt_apellidos.value.strip().replace(' ', '_')
            nuevo_nombre_archivo = f"{apellidos_nuevo}_{nombre_nuevo}"
            
            # Preparar nueva ubicación
            nuevo_grupo = self.dropdown_grupo.value
            nueva_practica = self.dropdown_practica.value
            
            nueva_carpeta = self.examenes_dir / nuevo_grupo
            
            # Solo añadir carpeta de práctica si no es una carpeta especial
            if nuevo_grupo not in ["extraviados", "problemático"]:
                nueva_carpeta = nueva_carpeta / f"Practica_{nueva_practica}"
            
            # Crear carpeta si no existe
            nueva_carpeta.mkdir(parents=True, exist_ok=True)
            
            # Nuevo path completo
            nuevo_path = nueva_carpeta / f"{nuevo_nombre_archivo}.pdf"
            
            # Evitar conflictos de nombres
            contador = 2
            while nuevo_path.exists() and nuevo_path != old_path:
                nuevo_path = nueva_carpeta / f"{nuevo_nombre_archivo}_{contador}.pdf"
                contador += 1
            
            # Mover archivo si es necesario
            if nuevo_path != old_path:
                shutil.move(str(old_path), str(nuevo_path))
                
                # Actualizar información en la lista
                current_exam['file_path'] = nuevo_path
                current_exam['grupo_actual'] = nuevo_grupo
                current_exam['practica'] = f"Practica_{nueva_practica}" if nuevo_grupo not in ["extraviados", "problemático"] else ""
                current_exam['nombre_archivo'] = nuevo_path.stem
                
                if nuevo_grupo in ["extraviados", "problemático"]:
                    self.status.value = f"✅ Movido a: {nuevo_grupo}/{nuevo_path.name}"
                else:
                    self.status.value = f"✅ Movido a: {nuevo_grupo}/Practica_{nueva_practica}/{nuevo_path.name}"
            else:
                self.status.value = "ℹ️ Sin cambios necesarios"
                
        except Exception as e:
            self.status.value = f"❌ Error aplicando cambios: {e}"

def iniciar_revision_examenes(examenes_dir="../data/examenes_procesados/"):
    """Inicia la interfaz de revisión de exámenes"""
    print("🚀 Iniciando revisor de exámenes...")
    reviewer = JupyterExamReviewer(examenes_dir)
    return reviewer

In [39]:
#reviewer = iniciar_revision_examenes()

In [40]:
import os
import shutil
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Combobox, Dropdown, HTML, ToggleButtons
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd
from difflib import get_close_matches
import re

# --- Cargar el DataFrame de alumnos (ajusta la ruta si es necesario) ---
students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
dfs = [pd.read_csv(filename) for filename in students_info]
df = pd.concat(dfs, ignore_index=True)
df["Nombre"] = df["Nombre"].str.strip().str.upper()
df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
df["Grupos"] = df["Grupos"].astype(str).str.strip()

# Lista de nombres completos para autocompletar
nombres_completos = [
    f"{row['Apellido(s)']} {row['Nombre']}" for _, row in df.iterrows()
]
nombre_a_grupo = {
    f"{row['Apellido(s)']} {row['Nombre']}": row['Grupos'] for _, row in df.iterrows()
}

class JupyterExamReviewer:
    def __init__(self, examenes_dir="../data/examenes_procesados/"):
        self.examenes_dir = Path(examenes_dir)
        self.current_index = 0
        self.exam_files = []
        self.out = Output()
        self.status = Label(value="Cargando exámenes...")

        # Cargar todos los archivos PDF
        self._load_exam_files()

        if not self.exam_files:
            self.status.value = "No se encontraron exámenes para revisar"
            return

        # Configurar widgets
        self._setup_widgets()
        self._show_current_exam()

    def _load_exam_files(self):
        print("🔍 Escaneando carpetas de exámenes...")
        for grupo_dir in self.examenes_dir.iterdir():
            if grupo_dir.is_dir():
                for practica_dir in grupo_dir.iterdir():
                    if practica_dir.is_dir():
                        for pdf_file in practica_dir.glob("*.pdf"):
                            self.exam_files.append({
                                'file_path': pdf_file,
                                'carpeta_actual': grupo_dir.name,
                                'practica': practica_dir.name if practica_dir.name.startswith("Practica_") else "",
                                'nombre_archivo': pdf_file.stem
                            })
                # También incluir PDFs sueltos en la carpeta (por si acaso)
                for pdf_file in grupo_dir.glob("*.pdf"):
                    self.exam_files.append({
                        'file_path': pdf_file,
                        'carpeta_actual': grupo_dir.name,
                        'practica': "",
                        'nombre_archivo': pdf_file.stem
                    })
        self.exam_files.sort(key=lambda x: (x['carpeta_actual'], x['nombre_archivo']))
        print(f"📁 Total de exámenes encontrados: {len(self.exam_files)}")

    def _setup_widgets(self):
        # Status (ruta de archivo) - arriba
        self.status.layout = Layout(width='100%')
        
        # Nombre del alumno (principal)
        self.combo_nombre = Combobox(
            placeholder='Escribe o selecciona...',
            options=nombres_completos,
            description='Alumno:',
            layout=Layout(width='450px')
        )
        self.combo_nombre.observe(self._on_nombre_change, names='value')

        # Botón de aplicar cambios
        self.btn_apply = Button(
            description='✓ Aplicar',
            button_style='success',
            layout=Layout(width='80px')
        )
        
        # Botón para eliminar archivo (NUEVO)
        self.btn_delete = Button(
            description='🗑️ Eliminar',
            button_style='danger',
            layout=Layout(width='90px')
        )
        
        # Contador de progreso
        self.progress_label = Label(value="", layout=Layout(width='140px'))

        # Widgets de grupo, práctica y carpeta
        grupos_unicos = sorted(df["Grupos"].unique())
        self.dropdown_grupo = Dropdown(
            options=grupos_unicos,
            description='Grupo:',
            layout=Layout(width='280px')  # Más ancho para que se vea bien
        )

        self.dropdown_practica = Dropdown(
            options=['2', '3', '4', '5'],
            description='Práctica:',
            layout=Layout(width='180px')
        )

        # Selector de carpeta destino
        carpetas_disponibles = [d.name for d in self.examenes_dir.iterdir() if d.is_dir()]
        self.dropdown_carpeta = Dropdown(
            options=carpetas_disponibles,
            description='Carpeta:',
            layout=Layout(width='170px')
        )
        self.dropdown_carpeta.observe(self._on_carpeta_change, names='value')

        # HTML para mostrar sugerencias de nombres similares (horizontal)
        self.similares_html = HTML(value="", layout=Layout(width='100%', max_height='60px'))

        # Botones de navegación 
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='120px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='120px'))
        
        # Selector de página ajustado (botones a la derecha del texto)
        self.page_label = Label(value="Ver pág:", layout=Layout(width='50px'))
        self.page_selector = ToggleButtons(
            options=[('1', 1), ('2', 2)],
            value=1,
            description='',  # Quitamos la descripción del control y la ponemos separada
            style={'button_width': '30px'},
            layout=Layout(width='80px')
        )
        
        self.page_selector.observe(self._on_page_change, names='value')

        # Conectar eventos
        self.btn_prev.on_click(lambda x: self._navigate(-1))
        self.btn_next.on_click(lambda x: self._navigate(1))
        self.btn_apply.on_click(lambda x: self._apply_changes())
        self.btn_delete.on_click(lambda x: self._delete_current_file())  # NUEVO

        # REORGANIZACIÓN DE LA INTERFAZ:
        
        # Fila 1: Path/status (menos importante, arriba)
        status_row = HBox([self.status])
        
        # Fila 2: Nombre + aplicar + eliminar + progreso (MODIFICADO)
        nombre_row = HBox([
            self.combo_nombre,
            self.btn_apply,
            self.btn_delete,  # NUEVO
            self.progress_label
        ])
        
        # Fila 3: Grupo, práctica, carpeta (más visibles)
        opciones_row = HBox([
            self.dropdown_grupo,
            self.dropdown_practica, 
            self.dropdown_carpeta
        ])
        
        # Fila 4: Sugerencias similares (a lo ancho)
        sugerencias_row = HBox([self.similares_html])
        
        # Fila 5: Navegación (abajo) + selector de páginas en línea
        page_selector_group = HBox([
            self.page_label, 
            self.page_selector
            ], 
            layout=Layout(width='140px')
        )
        nav_row = HBox([
            self.btn_prev,
            self.btn_next,
            page_selector_group
        ])  

        # Layout general
        self.interface = VBox([
            status_row,
            nombre_row,
            opciones_row,
            sugerencias_row,
            nav_row,
            self.out
        ])

        display(self.interface)

    def _delete_current_file(self):
        """Elimina el archivo actual (NUEVO MÉTODO)"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        file_path = current_exam['file_path']
        
        try:
            # Crear carpeta de eliminados si no existe
            deleted_folder = self.examenes_dir / "eliminados"
            deleted_folder.mkdir(exist_ok=True)
            
            # Mover a la carpeta de eliminados en lugar de eliminar permanentemente
            deleted_path = deleted_folder / file_path.name
            
            # Si ya existe en eliminados, añadir sufijo numérico
            contador = 2
            while deleted_path.exists():
                deleted_path = deleted_folder / f"{file_path.stem}_{contador}{file_path.suffix}"
                contador += 1
            
            # Mover archivo
            shutil.move(str(file_path), str(deleted_path))
            
            # Remover de la lista
            self.exam_files.pop(self.current_index)
            
            # Ajustar índice si es necesario
            if self.current_index >= len(self.exam_files):
                self.current_index = max(0, len(self.exam_files) - 1)
            
            self.status.value = f"🗑️ Eliminado: {file_path.name} → eliminados/{deleted_path.name}"
            
            # Mostrar siguiente examen o mensaje si no hay más
            if self.exam_files:
                self._show_current_exam()
            else:
                self.status.value = "🎉 No hay más exámenes para revisar"
                with self.out:
                    clear_output(wait=True)
                    print("No hay más exámenes para revisar")
                    
        except Exception as e:
            self.status.value = f"❌ Error eliminando archivo: {e}"

    def _on_carpeta_change(self, change):
        # Eliminado: Ya no se deshabilita el dropdown de práctica para ninguna carpeta
        pass

    def _on_nombre_change(self, change):
        valor = change['new']
        if valor in nombre_a_grupo:
            self.dropdown_grupo.value = nombre_a_grupo[valor]
            self.combo_nombre.value = valor
        self._update_similares(valor)

    def _on_page_change(self, change):
        self._show_exam_image(self.exam_files[self.current_index]['file_path'])

    def _update_similares(self, valor):
        if valor:
            matches = get_close_matches(valor, nombres_completos, n=4, cutoff=0)
            # Mostrar horizontalmente (una sola fila)
            html = "<b>Sugerencias:</b> "
            if matches:
                html += "<span style='display:inline-block;white-space:nowrap;'>"
                for i, m in enumerate(matches):
                    html += f"<span style='display:inline-block;margin-right:20px;font-size:90%'>{m} <span style='color:#888;font-size:85%'>({nombre_a_grupo.get(m, '-')})</span></span>"
                html += "</span>"
            else:
                html += "<i>No hay sugerencias</i>"
            self.similares_html.value = html
        else:
            self.similares_html.value = ""

    def _show_current_exam(self):
        if not self.exam_files:
            return

        current_exam = self.exam_files[self.current_index]
        self.progress_label.value = f"Examen {self.current_index + 1} de {len(self.exam_files)}"

        nombre_completo = current_exam['nombre_archivo'].replace('_', ' ')
        mejor_match = None
        for n in nombres_completos:
            if nombre_completo.upper() in n.upper():
                mejor_match = n
                break
        if mejor_match:
            self.combo_nombre.value = mejor_match
            self.dropdown_grupo.value = nombre_a_grupo[mejor_match]
        else:
            self.combo_nombre.value = nombre_completo
            self.dropdown_grupo.value = df["Grupos"].iloc[0]  # valor por defecto

        self._update_similares(self.combo_nombre.value)

        # Práctica
        if current_exam['practica']:
            practica_num = current_exam['practica'].replace('Practica_', '')
            if practica_num in ['2', '3', '4', '5']:
                self.dropdown_practica.value = practica_num
            else:
                self.dropdown_practica.value = '3'
        else:
            self.dropdown_practica.value = '3'

        # Carpeta actual
        self.dropdown_carpeta.value = current_exam['carpeta_actual']

        # Mostrar solo la página 1 por defecto
        self.page_selector.value = 1

        # Ajustar el rango del selector de página según el número de páginas
        reader = PdfReader(current_exam['file_path'])
        num_pages = len(reader.pages)
        if num_pages == 1:
            self.dropdown_carpeta.value = "problemático"
            self.page_selector.disabled = True
        else:
            self.page_selector.disabled = False

        # Eliminado: Ya no se deshabilita el dropdown de práctica

        if current_exam['practica']:
            self.status.value = f"📁 {current_exam['carpeta_actual']} / {current_exam['practica']} / {current_exam['nombre_archivo']}.pdf"
        else:
            self.status.value = f"📁 {current_exam['carpeta_actual']} / {current_exam['nombre_archivo']}.pdf"

        self._show_exam_image(current_exam['file_path'])

    def _show_exam_image(self, pdf_path):
        with self.out:
            clear_output(wait=True)
            try:
                reader = PdfReader(pdf_path)
                num_pages = len(reader.pages)
                page_num = self.page_selector.value
                if num_pages == 1:
                    page_num = 1
                images = convert_from_path(
                    pdf_path,
                    first_page=page_num,
                    last_page=page_num,
                    dpi=150,
                    fmt='jpeg'
                )
                if images:
                    image = images[0]
                    # Recortar más agresivamente por arriba y por abajo (1/4 de la altura)
                    cabecera_altura = int(min(450, image.height // 4))
                    # Recortar un poco desde arriba también (20 píxeles)
                    top_offset = 20
                    cropped = image.crop((0, top_offset, image.width, cabecera_altura))
                    plt.figure(figsize=(10, 6))  # Altura reducida
                    plt.imshow(cropped)
                    plt.axis('off')
                    plt.tight_layout()
                    plt.show()
                else:
                    print("❌ No se pudo cargar la imagen del PDF")
            except Exception as e:
                print(f"❌ Error cargando imagen: {e}")

    def _navigate(self, direction):
        if self._has_pending_changes():
            self._apply_changes()
        new_index = self.current_index + direction
        if 0 <= new_index < len(self.exam_files):
            self.current_index = new_index
            self._show_current_exam()
        elif new_index >= len(self.exam_files):
            self.status.value = "🎉 ¡Revisión completada! Has llegado al final."
        elif new_index < 0:
            self.status.value = "📍 Ya estás en el primer examen."

    def _has_pending_changes(self):
        if not self.exam_files:
            return False
        current_exam = self.exam_files[self.current_index]
        nombre_actual = self.combo_nombre.value.strip().upper().replace(' ', '_')
        nombre_archivo_actual = current_exam['nombre_archivo'].upper()
        carpeta_cambio = self.dropdown_carpeta.value != current_exam['carpeta_actual']
        practica_actual = current_exam['practica'].replace('Practica_', '') if current_exam['practica'] else ''
        # Eliminado: "and not self.dropdown_practica.disabled" ya que nunca se deshabilita
        practica_cambio = self.dropdown_practica.value != practica_actual
        grupo_cambio = self.dropdown_grupo.value != nombre_a_grupo.get(self.combo_nombre.value, self.dropdown_grupo.value)
        nombre_cambio = nombre_actual != nombre_archivo_actual
        return carpeta_cambio or practica_cambio or grupo_cambio or nombre_cambio

    def _apply_changes(self):
        if not self.exam_files:
            return
        current_exam = self.exam_files[self.current_index]
        old_path = current_exam['file_path']
        try:
            nombre_nuevo = self.combo_nombre.value.strip().replace(' ', '_')
            grupo_para_nombre = self.dropdown_grupo.value
            nueva_practica = self.dropdown_practica.value
            carpeta_destino = self.dropdown_carpeta.value

            # Nombre base: APELLIDOS_NOMBRE_P<num practica>_<Grupo>.pdf
            nombre_base = f"{nombre_nuevo}_P{nueva_practica}_{grupo_para_nombre}"
            nueva_carpeta = self.examenes_dir / carpeta_destino
            if carpeta_destino not in ["extraviados", "problemático"]:
                nueva_carpeta = nueva_carpeta / f"Practica_{nueva_practica}"
            nueva_carpeta.mkdir(parents=True, exist_ok=True)
            
            # COMPROBACIÓN MEJORADA DE ARCHIVOS EXISTENTES
            nuevo_path = nueva_carpeta / f"{nombre_base}.pdf"
            
            # Si el archivo destino es el mismo que el origen, no hacer nada
            if nuevo_path == old_path:
                self.status.value = "ℹ️ Sin cambios necesarios"
                return
            
            # Si existe otro archivo con el mismo nombre, buscar sufijo disponible
            contador = 2
            while nuevo_path.exists():
                # Extraer cualquier sufijo numérico existente del nombre base
                match = re.search(r'_(\d+)$', nombre_base)
                if match:
                    # Ya tiene sufijo numérico, incrementarlo
                    numero_actual = int(match.group(1))
                    nombre_sin_sufijo = nombre_base[:match.start()]
                    nuevo_nombre_base = f"{nombre_sin_sufijo}_{numero_actual + contador - 1}"
                else:
                    # No tiene sufijo, añadir uno
                    nuevo_nombre_base = f"{nombre_base}_{contador}"
                
                nuevo_path = nueva_carpeta / f"{nuevo_nombre_base}.pdf"
                contador += 1
                
                # Seguridad: evitar bucle infinito
                if contador > 100:
                    self.status.value = "❌ Error: Demasiados archivos duplicados"
                    return
            
            # Realizar el movimiento de archivo
            try:
                shutil.move(str(old_path), str(nuevo_path))
                
                # Actualizar información en la lista
                current_exam['file_path'] = nuevo_path
                current_exam['carpeta_actual'] = carpeta_destino
                current_exam['practica'] = f"Practica_{nueva_practica}" if carpeta_destino not in ["extraviados", "problemático"] else ""
                current_exam['nombre_archivo'] = nuevo_path.stem
                
                # Mensaje de confirmación
                if carpeta_destino in ["extraviados", "problemático"]:
                    self.status.value = f"✅ Movido a: {carpeta_destino}/{nuevo_path.name}"
                else:
                    self.status.value = f"✅ Movido a: {carpeta_destino}/Practica_{nueva_practica}/{nuevo_path.name}"
                    
                # Si se añadió un sufijo, avisar
                if contador > 2:
                    self.status.value += f" (duplicado evitado con sufijo _{contador-2})"
                    
            except PermissionError:
                self.status.value = "❌ Error: Archivo en uso o sin permisos"
            except FileNotFoundError:
                self.status.value = "❌ Error: Archivo origen no encontrado"
            except Exception as move_error:
                self.status.value = f"❌ Error moviendo archivo: {move_error}"
                
        except Exception as e:
            self.status.value = f"❌ Error aplicando cambios: {e}"

def iniciar_revision_examenes(examenes_dir="../data/examenes_procesados/"):
    print("🚀 Iniciando revisor de exámenes...")
    reviewer = JupyterExamReviewer(examenes_dir)
    return reviewer

In [41]:
# Para lanzar la interfaz:
#reviewer = iniciar_revision_examenes()

In [42]:
import zipfile
from datetime import datetime  # AÑADIR ESTE IMPORT

def crear_backup_examenes(carpeta_examenes="../data/examenes_procesados/", carpeta_backup="../data/"):
    """
    Crea un backup completo de la carpeta de exámenes procesados
    """
    carpeta_examenes = Path(carpeta_examenes)
    carpeta_backup = Path(carpeta_backup)
    
    # Crear nombre del backup con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_backup = f"examenes_procesados_backup_{timestamp}.zip"
    ruta_backup = carpeta_backup / nombre_backup
    
    print(f"🔄 Creando backup de: {carpeta_examenes}")
    print(f"📦 Archivo de backup: {ruta_backup}")
    
    try:
        # Crear el archivo ZIP
        with zipfile.ZipFile(ruta_backup, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Recorrer todos los archivos y carpetas
            for root, dirs, files in os.walk(carpeta_examenes):
                for file in files:
                    file_path = Path(root) / file
                    # Calcular la ruta relativa para mantener la estructura
                    arcname = file_path.relative_to(carpeta_examenes.parent)
                    zipf.write(file_path, arcname)
                    print(f"  ✓ Añadido: {arcname}")
        
        print(f"\n✅ Backup completado exitosamente!")
        print(f"📁 Tamaño del backup: {ruta_backup.stat().st_size / (1024*1024):.2f} MB")
        print(f"💾 Ubicación: {ruta_backup}")
        
        return str(ruta_backup)
        
    except Exception as e:
        print(f"❌ Error creando backup: {e}")
        return None

In [43]:
crear_backup_examenes()

🔄 Creando backup de: ..\data\examenes_procesados
📦 Archivo de backup: ..\data\examenes_procesados_backup_20250611_144747.zip
  ✓ Añadido: examenes_procesados\errores_openai.csv
  ✓ Añadido: examenes_procesados\seguimiento_examenes.csv
  ✓ Añadido: examenes_procesados\CITIM11\Practica_2\PARIENTE_CARRIAZO_ANTONIO_P2_CITIM11.pdf
  ✓ Añadido: examenes_procesados\CITIM11\Practica_3\ARIAS_CASADO_ALBA_P3_CITIM11.pdf
  ✓ Añadido: examenes_procesados\CITIM11\Practica_3\CARMONA_OCAÑA_DANIEL_P3_CITIM11.pdf
  ✓ Añadido: examenes_procesados\CITIM11\Practica_3\DE_LA_IGLESIA_NUÑEZ_PEDRO_JOSE_P3_CITIM11.pdf
  ✓ Añadido: examenes_procesados\CITIM11\Practica_3\EXPOSITO_SONO_HARITZ_ENDIKA_P3_CITIM11.pdf
  ✓ Añadido: examenes_procesados\CITIM11\Practica_3\FERNANDEZ_NIETO_DAVID_P3_CITIM11.pdf
  ✓ Añadido: examenes_procesados\CITIM11\Practica_3\GARCIA_FERNANDEZ_LORENZO_P3_CITIM11.pdf
  ✓ Añadido: examenes_procesados\CITIM11\Practica_3\HUERGA_GIL_JAVIER_P3_CITIM11.pdf
  ✓ Añadido: examenes_procesados\CITIM11

'..\\data\\examenes_procesados_backup_20250611_144747.zip'

In [44]:
import os
from pathlib import Path

def eliminar_carpetas_vacias(ruta_base="../data/examenes_procesados/"):
    """
    Elimina recursivamente todas las carpetas vacías en la ruta especificada
    """
    ruta_base = Path(ruta_base)
    
    if not ruta_base.exists():
        print(f"❌ La ruta {ruta_base} no existe")
        return
    
    carpetas_eliminadas = []
    
    # Función recursiva para eliminar carpetas vacías
    def eliminar_vacias_recursivo(directorio):
        """Elimina carpetas vacías de forma recursiva, empezando por las más profundas"""
        try:
            # Primero procesar subdirectorios
            for item in directorio.iterdir():
                if item.is_dir():
                    eliminar_vacias_recursivo(item)
            
            # Luego verificar si el directorio actual está vacío
            if directorio.is_dir() and not any(directorio.iterdir()):
                directorio.rmdir()
                carpetas_eliminadas.append(str(directorio))
                print(f"🗑️ Eliminada carpeta vacía: {directorio}")
                
        except PermissionError:
            print(f"⚠️ Sin permisos para eliminar: {directorio}")
        except OSError as e:
            print(f"⚠️ Error eliminando {directorio}: {e}")
    
    print(f"🔍 Buscando carpetas vacías en: {ruta_base}")
    print("=" * 50)
    
    # Ejecutar la eliminación recursiva
    eliminar_vacias_recursivo(ruta_base)
    
    print("=" * 50)
    print(f"✅ Proceso completado")
    print(f"📊 Total de carpetas vacías eliminadas: {len(carpetas_eliminadas)}")
    
    if carpetas_eliminadas:
        print("\n📋 Carpetas eliminadas:")
        for carpeta in carpetas_eliminadas:
            print(f"  - {carpeta}")
    else:
        print("ℹ️ No se encontraron carpetas vacías para eliminar")
    
    return carpetas_eliminadas


In [45]:

# Ejecutar la limpieza
#eliminar_carpetas_vacias()

In [46]:
import pandas as pd
import os
import re
from pathlib import Path
import unicodedata
from rapidfuzz import fuzz

def limpiar_texto_busqueda(texto):
    """Normaliza texto para búsquedas flexibles"""
    if not texto:
        return ""
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')
    texto = texto.upper().strip()
    texto = re.sub(r'[^A-Z0-9\s]', '', texto)
    return texto

def buscar_examenes_en_carpetas(df_con_practicas, ruta_examenes="../data/examenes_procesados/"):
    """
    Busca exámenes en todas las carpetas (incluyendo problemático) y actualiza el DataFrame
    """
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"❌ La ruta {ruta_examenes} no existe")
        return df_con_practicas
    
    # Crear copia del DataFrame para no modificar el original
    df_resultado = df_con_practicas.copy()
    
    # Añadir columnas de exámenes si no existen
    if 'Examen_3' not in df_resultado.columns:
        df_resultado['Examen_3'] = 0
        df_resultado['Comentario_Examen_3'] = 'PNP'
    if 'Examen_5' not in df_resultado.columns:
        df_resultado['Examen_5'] = 0
        df_resultado['Comentario_Examen_5'] = 'PNP'
    
    # Resetear columnas de exámenes
    df_resultado['Examen_3'] = 0
    df_resultado['Comentario_Examen_3'] = 'PNP'
    df_resultado['Examen_5'] = 0
    df_resultado['Comentario_Examen_5'] = 'PNP'
    
    examenes_encontrados = []
    
    print("🔍 Escaneando carpetas de exámenes...")
    print("=" * 60)
    
    # Recorrer todas las carpetas
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir():
            continue
            
        print(f"\n📁 Revisando carpeta: {carpeta_grupo.name}")
        
        # Si es una carpeta de grupo normal (con subcarpetas de prácticas)
        if carpeta_grupo.name not in ["extraviados", "problemático", "eliminados"]:
            for subcarpeta in carpeta_grupo.iterdir():
                if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                    # Extraer número de práctica
                    practica_num = subcarpeta.name.replace("Practica_", "")
                    if practica_num in ['3', '5']:
                        print(f"  📂 {subcarpeta.name}")
                        procesar_archivos_practica(subcarpeta, practica_num, examenes_encontrados, carpeta_grupo.name)
        
        # Si es una carpeta especial (problemático, extraviados)
        else:
            print(f"  📂 Carpeta especial: {carpeta_grupo.name}")
            procesar_archivos_especiales(carpeta_grupo, examenes_encontrados)
    
    print("\n" + "=" * 60)
    print("🔄 Actualizando DataFrame con exámenes encontrados...")
    
    # Actualizar DataFrame con los exámenes encontrados
    examenes_marcados = actualizar_dataframe_examenes(df_resultado, examenes_encontrados)
    
    print("\n📊 RESUMEN FINAL:")
    print(f"Total de archivos PDF encontrados: {len(examenes_encontrados)}")
    print(f"Exámenes marcados en DataFrame: {examenes_marcados}")
    
    # Mostrar estadísticas por práctica
    total_examen_3 = df_resultado['Examen_3'].sum()
    total_examen_5 = df_resultado['Examen_5'].sum()
    total_alumnos = len(df_resultado)
    
    print(f"\n📈 ESTADÍSTICAS:")
    print(f"Examen Práctica 3: {total_examen_3}/{total_alumnos} ({total_examen_3/total_alumnos*100:.1f}%)")
    print(f"Examen Práctica 5: {total_examen_5}/{total_alumnos} ({total_examen_5/total_alumnos*100:.1f}%)")
    
    # Mostrar estadísticas por grupo
    if 'Grupos' in df_resultado.columns:
        print(f"\n📋 ESTADÍSTICAS POR GRUPO:")
        resumen_grupos = df_resultado.groupby('Grupos').agg({
            'Examen_3': 'sum',
            'Examen_5': 'sum'
        })
        print(resumen_grupos)
    
    return df_resultado

def procesar_archivos_practica(carpeta_practica, practica_num, examenes_encontrados, grupo):
    """Procesa archivos en una carpeta de práctica específica"""
    for archivo_pdf in carpeta_practica.glob("*.pdf"):
        info_examen = extraer_info_nombre_archivo(archivo_pdf, practica_num, grupo)
        examenes_encontrados.append(info_examen)
        print(f"    ✓ {archivo_pdf.name} → {info_examen['apellidos']} {info_examen['nombre']}")

def procesar_archivos_especiales(carpeta_especial, examenes_encontrados):
    """Procesa archivos en carpetas especiales (problemático, extraviados)"""
    for archivo_pdf in carpeta_especial.glob("*.pdf"):
        # Intentar detectar práctica del nombre del archivo o contenido
        practica_detectada = detectar_practica_archivo(archivo_pdf)
        if practica_detectada in ['3', '5']:
            info_examen = extraer_info_nombre_archivo(archivo_pdf, practica_detectada, carpeta_especial.name)
            examenes_encontrados.append(info_examen)
            print(f"    ⚠️ {archivo_pdf.name} → {info_examen['apellidos']} {info_examen['nombre']} (P{practica_detectada})")

def detectar_practica_archivo(archivo_pdf):
    """Detecta el número de práctica desde el nombre del archivo"""
    nombre = archivo_pdf.name.upper()
    
    # Buscar patrones como P3, P5, _3_, _5_, etc.
    patrones_practica = [
        r'P(\d)',
        r'_(\d)_',
        r'PRACTICA_?(\d)',
        r'LISTA.*3',  # Para práctica de listas
        r'GRAFO.*5'   # Para práctica de grafos
    ]
    
    for patron in patrones_practica:
        match = re.search(patron, nombre)
        if match:
            if 'LISTA' in nombre or match.group(1) == '3':
                return '3'
            elif 'GRAFO' in nombre or match.group(1) == '5':
                return '5'
    
    return 'desconocida'

def extraer_info_nombre_archivo(archivo_pdf, practica_num, grupo):
    """Extrae información del nombre del archivo"""
    nombre_archivo = archivo_pdf.stem
    
    # Patrones comunes: APELLIDOS_NOMBRE, APELLIDOS_NOMBRE_P3_GRUPO, etc.
    partes = nombre_archivo.replace('_', ' ').split()
    
    # Limpiar partes que no son nombres (números de práctica, grupos, etc.)
    partes_limpias = []
    for parte in partes:
        # Saltar si es un grupo conocido, número de práctica, etc.
        if not (parte.startswith('P') and parte[1:].isdigit()) and \
           not parte.isdigit() and \
           not parte in ['CITIM11', 'CITIM12', 'IWSIM11', 'IWSIM12', 'CITIT11', 'IWSIT11', 'IWSIT12']:
            partes_limpias.append(parte)
    
    # Asumir que las primeras partes son apellidos y la última es nombre
    if len(partes_limpias) >= 2:
        apellidos = ' '.join(partes_limpias[:-1])
        nombre = partes_limpias[-1]
    elif len(partes_limpias) == 1:
        apellidos = partes_limpias[0]
        nombre = ''
    else:
        apellidos = nombre_archivo
        nombre = ''
    
    return {
        'archivo': archivo_pdf.name,
        'ruta': str(archivo_pdf),
        'apellidos': limpiar_texto_busqueda(apellidos),
        'nombre': limpiar_texto_busqueda(nombre),
        'practica': practica_num,
        'grupo_carpeta': grupo
    }

def actualizar_dataframe_examenes(df_resultado, examenes_encontrados):
    """Actualiza el DataFrame marcando los exámenes encontrados"""
    examenes_marcados = 0
    
    for examen in examenes_encontrados:
        if examen['practica'] not in ['3', '5']:
            continue
            
        apellidos_examen = examen['apellidos']
        nombre_examen = examen['nombre']
        practica = examen['practica']
        
        # Buscar coincidencia en el DataFrame usando fuzzy matching
        mejor_coincidencia = None
        mejor_score = 0
        
        for idx, row in df_resultado.iterrows():
            apellidos_df = limpiar_texto_busqueda(str(row['Apellido(s)']))
            nombre_df = limpiar_texto_busqueda(str(row['Nombre']))
            
            # Calcular similitud
            score_apellidos = fuzz.ratio(apellidos_examen, apellidos_df) if apellidos_examen else 0
            score_nombre = fuzz.ratio(nombre_examen, nombre_df) if nombre_examen else 0
            
            # Score combinado (dar más peso a apellidos)
            score_total = (score_apellidos * 0.7 + score_nombre * 0.3)
            
            if score_total > mejor_score and score_total >= 70:  # Umbral del 70%
                mejor_score = score_total
                mejor_coincidencia = idx
        
        # Marcar en el DataFrame si encontramos coincidencia
        if mejor_coincidencia is not None:
            df_resultado.loc[mejor_coincidencia, f'Examen_{practica}'] = 1
            df_resultado.loc[mejor_coincidencia, f'Comentario_Examen_{practica}'] = f'Encontrado ({mejor_score:.1f}%)'
            examenes_marcados += 1
            
            # Mostrar la coincidencia
            row_matched = df_resultado.loc[mejor_coincidencia]
            print(f"    ✅ Coincidencia: {apellidos_examen} {nombre_examen} → {row_matched['Apellido(s)']} {row_matched['Nombre']} ({mejor_score:.1f}%)")
        else:
            print(f"    ❌ Sin coincidencia: {apellidos_examen} {nombre_examen}")
    
    return examenes_marcados

# Cargar DataFrame si no existe
if 'df_con_practicas' not in globals():
    print("📄 Cargando DataFrame de alumnos...")
    students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
    dfs = [pd.read_csv(filename) for filename in students_info]
    df = pd.concat(dfs, ignore_index=True)
    df["Nombre"] = df["Nombre"].str.strip().str.upper()
    df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
    
    # Verificar prácticas entregadas primero
    from pathlib import Path
    import zipfile
    import unicodedata
    import re

    def normalizar_texto(texto):
        """Normaliza texto eliminando acentos y caracteres especiales"""
        texto = unicodedata.normalize('NFD', texto)
        texto = ''.join(char for char in texto if unicodedata.category(char) != 'Mn')
        texto = texto.upper().strip()
        texto = re.sub(r'[^A-Z0-9\s]', '', texto)
        return texto

    def buscar_practica_en_zips(apellidos, nombre, practica_num=3, ruta_data="./../data/"):
        """Busca si existe una práctica para un alumno en los archivos ZIP"""
        ruta_practica = Path(ruta_data) / f"Practica{practica_num}"
        
        if not ruta_practica.exists():
            return False
        
        apellidos_norm = normalizar_texto(apellidos)
        nombre_norm = normalizar_texto(nombre)
        
        for archivo_zip in ruta_practica.glob("*.zip"):
            try:
                with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
                    for archivo in zip_ref.namelist():
                        archivo_norm = normalizar_texto(archivo)
                        if apellidos_norm in archivo_norm and nombre_norm in archivo_norm:
                            return True
            except Exception as e:
                continue
        return False

    # Crear DataFrame con prácticas
    df_con_practicas = df.copy()
    df_con_practicas['Presentada_3'] = 0
    df_con_practicas['Comentario_3'] = 'NP'
    df_con_practicas['Presentada_5'] = 0
    df_con_practicas['Comentario_5'] = 'NP'
    
    for idx, row in df_con_practicas.iterrows():
        nombre = str(row['Nombre'])
        apellidos = str(row['Apellido(s)'])
        
        if buscar_practica_en_zips(apellidos, nombre, 3):
            df_con_practicas.loc[idx, 'Presentada_3'] = 1
            df_con_practicas.loc[idx, 'Comentario_3'] = ''
        
        if buscar_practica_en_zips(apellidos, nombre, 5):
            df_con_practicas.loc[idx, 'Presentada_5'] = 1
            df_con_practicas.loc[idx, 'Comentario_5'] = ''

In [53]:
# Ejecutar la búsqueda de exámenes
df_con_practicas_y_examenes = buscar_examenes_en_carpetas(df_con_practicas)

#print("\n🎯 PROCESO COMPLETADO")
print("DataFrame actualizado con información de prácticas entregadas Y exámenes realizados")

🔍 Escaneando carpetas de exámenes...

📁 Revisando carpeta: CITIM11
  📂 Practica_3
    ✓ ARIAS_CASADO_ALBA_P3_CITIM11.pdf → ARIAS CASADO ALBA
    ✓ CARMONA_OCAÑA_DANIEL_P3_CITIM11.pdf → CARMONA OCANA DANIEL
    ✓ DE_LA_IGLESIA_NUÑEZ_PEDRO_JOSE_P3_CITIM11.pdf → DE LA IGLESIA NUNEZ PEDRO JOSE
    ✓ EXPOSITO_SONO_HARITZ_ENDIKA_P3_CITIM11.pdf → EXPOSITO SONO HARITZ ENDIKA
    ✓ FERNANDEZ_NIETO_DAVID_P3_CITIM11.pdf → FERNANDEZ NIETO DAVID
    ✓ GARCIA_FERNANDEZ_LORENZO_P3_CITIM11.pdf → GARCIA FERNANDEZ LORENZO
    ✓ HUERGA_GIL_JAVIER_P3_CITIM11.pdf → HUERGA GIL JAVIER
    ✓ LUCERO_PRADA_IRENE_P3_CITIM11.pdf → LUCERO PRADA IRENE
    ✓ MARTIN_VERDUGO_CRISTINA_P3_CITIM11.pdf → MARTIN VERDUGO CRISTINA
    ✓ MASSERA_SALCEDO_GUILLERMO_P3_CITIM11.pdf → MASSERA SALCEDO GUILLERMO
    ✓ MORAN_RUIZ_JAIME_P3_CITIM11.pdf → MORAN RUIZ JAIME
    ✓ NAZARENKO_KSENIA_P3_CITIM11.pdf → NAZARENKO KSENIA
    ✓ NIETO_HERNANDEZ_JAVIER_P3_CITIM11.pdf → NIETO HERNANDEZ JAVIER
    ✓ PLAZA_PASCUAL_LUCAS_P3_CITIM11.pdf 

In [48]:
#display(df_con_practicas_y_examenes.head(10))

In [49]:
def estandarizar_nombres_examenes(ruta_examenes="../data/examenes_procesados/"):
    """
    Verifica y estandariza todos los nombres de archivos de exámenes 
    siguiendo el formato: APELLIDOS_NOMBRE_P<numpractica>_<GRUPO>
    """
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"❌ La ruta {ruta_examenes} no existe")
        return
    
    archivos_procesados = 0
    archivos_renombrados = 0
    errores = []
    
    print("🔍 Verificando nombres de archivos en carpetas de exámenes...")
    print("=" * 80)
    
    # Recorrer todas las carpetas de grupos
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir():
            continue
            
        print(f"\n📁 Procesando carpeta: {carpeta_grupo.name}")
        
        # Si es una carpeta de grupo normal (con subcarpetas de prácticas)
        if carpeta_grupo.name not in ["extraviados", "problemático", "eliminados"]:
            for subcarpeta in carpeta_grupo.iterdir():
                if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                    # Extraer número de práctica
                    practica_num = subcarpeta.name.replace("Practica_", "")
                    print(f"  📂 {subcarpeta.name}")
                    
                    # Procesar archivos en esta carpeta
                    for archivo_pdf in subcarpeta.glob("*.pdf"):
                        archivos_procesados += 1
                        
                        nombre_actual = archivo_pdf.stem
                        formato_esperado = generar_nombre_estandar(
                            nombre_actual, practica_num, carpeta_grupo.name
                        )
                        
                        if nombre_actual != formato_esperado:
                            print(f"    🔄 Renombrando:")
                            print(f"       De: {nombre_actual}.pdf")
                            print(f"       A:  {formato_esperado}.pdf")
                            
                            try:
                                nuevo_path = archivo_pdf.parent / f"{formato_esperado}.pdf"
                                
                                # Evitar conflictos
                                contador = 2
                                while nuevo_path.exists():
                                    nuevo_formato = f"{formato_esperado}_{contador}"
                                    nuevo_path = archivo_pdf.parent / f"{nuevo_formato}.pdf"
                                    print(f"       ⚠️  Conflicto detectado, usando: {nuevo_formato}.pdf")
                                    contador += 1
                                
                                archivo_pdf.rename(nuevo_path)
                                archivos_renombrados += 1
                                print(f"       ✅ Renombrado exitosamente")
                                
                            except Exception as e:
                                error_msg = f"Error renombrando {archivo_pdf.name}: {e}"
                                errores.append(error_msg)
                                print(f"       ❌ {error_msg}")
                        else:
                            print(f"    ✅ {nombre_actual}.pdf (ya tiene formato correcto)")
        
        # Procesar carpetas especiales
        else:
            print(f"  📂 Carpeta especial: {carpeta_grupo.name}")
            for archivo_pdf in carpeta_grupo.glob("*.pdf"):
                archivos_procesados += 1
                
                # Para carpetas especiales, intentar detectar práctica y grupo del nombre
                practica_detectada = detectar_practica_del_nombre(archivo_pdf.name)
                grupo_detectado = detectar_grupo_del_nombre(archivo_pdf.name)
                
                if practica_detectada and grupo_detectado:
                    nombre_actual = archivo_pdf.stem
                    formato_esperado = generar_nombre_estandar(
                        nombre_actual, practica_detectada, grupo_detectado
                    )
                    
                    if nombre_actual != formato_esperado:
                        print(f"    🔄 Renombrando (carpeta especial):")
                        print(f"       De: {nombre_actual}.pdf")
                        print(f"       A:  {formato_esperado}.pdf")
                        
                        try:
                            nuevo_path = archivo_pdf.parent / f"{formato_esperado}.pdf"
                            
                            # Evitar conflictos
                            contador = 2
                            while nuevo_path.exists():
                                nuevo_formato = f"{formato_esperado}_{contador}"
                                nuevo_path = archivo_pdf.parent / f"{nuevo_formato}.pdf"
                                contador += 1
                            
                            archivo_pdf.rename(nuevo_path)
                            archivos_renombrados += 1
                            print(f"       ✅ Renombrado exitosamente")
                            
                        except Exception as e:
                            error_msg = f"Error renombrando {archivo_pdf.name}: {e}"
                            errores.append(error_msg)
                            print(f"       ❌ {error_msg}")
                    else:
                        print(f"    ✅ {archivo_pdf.name} (ya tiene formato correcto)")
                else:
                    print(f"    ⚠️  {archivo_pdf.name} (no se pudo detectar práctica/grupo)")
    
    print("\n" + "=" * 80)
    print("📊 RESUMEN DE ESTANDARIZACIÓN:")
    print(f"Total de archivos procesados: {archivos_procesados}")
    print(f"Archivos renombrados: {archivos_renombrados}")
    print(f"Archivos que ya tenían formato correcto: {archivos_procesados - archivos_renombrados}")
    
    if errores:
        print(f"\n❌ ERRORES ({len(errores)}):")
        for error in errores:
            print(f"  - {error}")
    else:
        print("\n✅ No se encontraron errores")

def generar_nombre_estandar(nombre_actual, practica_num, grupo):
    """
    Genera el nombre estándar basado en el nombre actual
    Formato: APELLIDOS_NOMBRE_P<numpractica>_<GRUPO>
    """
    # Limpiar el nombre actual de elementos no deseados
    nombre_limpio = nombre_actual.upper()
    
    # Remover elementos que no sean nombres (práctica anterior, grupo anterior, etc.)
    elementos_a_remover = [
        rf'_P\d+', rf'P\d+_', rf'P\d+$',  # Prácticas
        rf'_{grupo}', rf'{grupo}_', rf'^{grupo}',  # Grupo actual
        r'_CITIM\d+', r'_IWSIM\d+', r'_CITIT\d+', r'_IWSIT\d+',  # Otros grupos
        r'CITIM\d+_', r'IWSIM\d+_', r'CITIT\d+_', r'IWSIT\d+_',
        r'_\d+$'  # Números al final
    ]
    
    for patron in elementos_a_remover:
        nombre_limpio = re.sub(patron, '', nombre_limpio)
    
    # Limpiar guiones bajos múltiples y al inicio/final
    nombre_limpio = re.sub(r'_+', '_', nombre_limpio).strip('_')
    
    # Si está vacío, usar un nombre por defecto
    if not nombre_limpio:
        nombre_limpio = "SIN_NOMBRE"
    
    # Construir el formato estándar
    formato_estandar = f"{nombre_limpio}_P{practica_num}_{grupo}"
    
    return formato_estandar

def detectar_practica_del_nombre(nombre_archivo):
    """Detecta el número de práctica del nombre del archivo"""
    nombre = nombre_archivo.upper()
    
    # Buscar patrones de práctica
    patrones = [
        r'P(\d)',
        r'_(\d)_',
        r'PRACTICA_?(\d)',
    ]
    
    for patron in patrones:
        match = re.search(patron, nombre)
        if match:
            return match.group(1)
    
    # Buscar palabras clave
    if 'LISTA' in nombre or '3' in nombre:
        return '3'
    elif 'GRAFO' in nombre or '5' in nombre:
        return '5'
    elif '4' in nombre:
        return '4'
    elif '2' in nombre:
        return '2'
    
    return None

def detectar_grupo_del_nombre(nombre_archivo):
    """Detecta el grupo del nombre del archivo"""
    nombre = nombre_archivo.upper()
    
    # Buscar patrones de grupo
    grupos_posibles = ['CITIM11', 'CITIM12', 'IWSIM11', 'IWSIM12', 'CITIT11', 'CITIT12', 'IWSIT11', 'IWSIT12']
    
    for grupo in grupos_posibles:
        if grupo in nombre:
            return grupo
    
    return None

def verificar_estructura_completa(ruta_examenes="../data/examenes_procesados/"):
    """
    Verifica que todos los archivos sigan el formato correcto después de la estandarización
    """
    ruta_examenes = Path(ruta_examenes)
    
    print("\n🔍 VERIFICACIÓN FINAL DE FORMATO:")
    print("=" * 60)
    
    patron_correcto = re.compile(r'^.+_P[2-5]_[A-Z]+\d+$')
    archivos_incorrectos = []
    
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir() or carpeta_grupo.name in ["eliminados"]:
            continue
            
        if carpeta_grupo.name not in ["extraviados", "problemático"]:
            # Carpetas de grupo normales
            for subcarpeta in carpeta_grupo.iterdir():
                if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                    for archivo_pdf in subcarpeta.glob("*.pdf"):
                        nombre = archivo_pdf.stem
                        if not patron_correcto.match(nombre):
                            archivos_incorrectos.append(str(archivo_pdf.relative_to(ruta_examenes)))
                        else:
                            print(f"✅ {archivo_pdf.relative_to(ruta_examenes)}")
        else:
            # Carpetas especiales
            for archivo_pdf in carpeta_grupo.glob("*.pdf"):
                nombre = archivo_pdf.stem
                if not patron_correcto.match(nombre):
                    archivos_incorrectos.append(str(archivo_pdf.relative_to(ruta_examenes)))
                else:
                    print(f"✅ {archivo_pdf.relative_to(ruta_examenes)}")
    
    if archivos_incorrectos:
        print(f"\n⚠️  ARCHIVOS QUE AÚN NO SIGUEN EL FORMATO:")
        for archivo in archivos_incorrectos:
            print(f"  - {archivo}")
    else:
        print(f"\n🎉 ¡Todos los archivos siguen el formato correcto!")

# Ejecutar la estandarización
estandarizar_nombres_examenes()

# Verificar el resultado
#verificar_estructura_completa()

🔍 Verificando nombres de archivos en carpetas de exámenes...

📁 Procesando carpeta: CITIM11
  📂 Practica_2
    ✅ PARIENTE_CARRIAZO_ANTONIO_P2_CITIM11.pdf (ya tiene formato correcto)
  📂 Practica_3
    ✅ ARIAS_CASADO_ALBA_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ CARMONA_OCAÑA_DANIEL_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ DE_LA_IGLESIA_NUÑEZ_PEDRO_JOSE_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ EXPOSITO_SONO_HARITZ_ENDIKA_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ FERNANDEZ_NIETO_DAVID_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ GARCIA_FERNANDEZ_LORENZO_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ HUERGA_GIL_JAVIER_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ LUCERO_PRADA_IRENE_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ MARTIN_VERDUGO_CRISTINA_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ MASSERA_SALCEDO_GUILLERMO_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ MORAN_RUIZ_JAIME_P3_CITIM11.pdf (ya tiene formato correcto)
    ✅ NAZARENKO

In [50]:
import hashlib
import os
from pathlib import Path

def encontrar_archivos_con_sufijo_numerico(ruta_examenes="../data/examenes_procesados/"):
    """
    Encuentra y lista todos los archivos que tienen sufijos numéricos (_2, _3, etc.)
    """
    
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"❌ La ruta {ruta_examenes} no existe")
        return []
    
    # Patrón para detectar archivos con sufijo numérico
    patron_sufijo = re.compile(r'.*_(\d+)\.pdf$', re.IGNORECASE)
    
    archivos_con_sufijo = []
    archivos_procesados = 0
    
    print("🔍 Buscando archivos con sufijos numéricos (_N)...")
    print("=" * 60)
    
    # Recorrer todas las carpetas y archivos
    for root, dirs, files in os.walk(ruta_examenes):
        for file in files:
            if file.lower().endswith('.pdf'):
                archivos_procesados += 1
                
                # Verificar si el archivo tiene sufijo numérico
                match = patron_sufijo.match(file)
                if match:
                    archivo_path = Path(root) / file
                    sufijo_numero = int(match.group(1))
                    
                    info_archivo = {
                        'ruta_completa': archivo_path,
                        'ruta_relativa': archivo_path.relative_to(ruta_examenes),
                        'nombre': file,
                        'sufijo': sufijo_numero,
                        'carpeta': archivo_path.parent.name,
                        'tamaño': archivo_path.stat().st_size
                    }
                    
                    archivos_con_sufijo.append(info_archivo)
    
    print(f"\n📊 RESUMEN:")
    print(f"Total de archivos procesados: {archivos_procesados}")
    print(f"Archivos con sufijo numérico encontrados: {len(archivos_con_sufijo)}")
    
    if archivos_con_sufijo:
        print(f"\n📁 ARCHIVOS CON SUFIJOS NUMÉRICOS:")
        print("=" * 80)
        
        # Ordenar por carpeta y luego por nombre
        archivos_ordenados = sorted(archivos_con_sufijo, key=lambda x: (str(x['ruta_relativa'].parent), x['nombre']))
        
        carpeta_actual = None
        for archivo in archivos_ordenados:
            carpeta = str(archivo['ruta_relativa'].parent)
            
            # Mostrar encabezado de carpeta si cambia
            if carpeta != carpeta_actual:
                print(f"\n📂 {carpeta}")
                carpeta_actual = carpeta
            
            # Mostrar archivo con sufijo resaltado
            nombre_sin_sufijo = archivo['nombre'].replace(f"_{archivo['sufijo']}.pdf", ".pdf")
            print(f"   ✓ {archivo['nombre']} (sufijo: _{archivo['sufijo']})")
            print(f"     → Nombre original sería: {nombre_sin_sufijo}")
        
        # Estadísticas por sufijo
        print(f"\n📈 ESTADÍSTICAS POR SUFIJO:")
        sufijos_count = {}
        for archivo in archivos_con_sufijo:
            sufijo = archivo['sufijo']
            if sufijo not in sufijos_count:
                sufijos_count[sufijo] = 0
            sufijos_count[sufijo] += 1
        
        for sufijo in sorted(sufijos_count.keys()):
            print(f"   Sufijo _{sufijo}: {sufijos_count[sufijo]} archivos")
        
        # Estadísticas por carpeta
        print(f"\n📂 ESTADÍSTICAS POR CARPETA:")
        carpetas_count = {}
        for archivo in archivos_con_sufijo:
            carpeta = str(archivo['ruta_relativa'].parent)
            if carpeta not in carpetas_count:
                carpetas_count[carpeta] = 0
            carpetas_count[carpeta] += 1
        
        for carpeta in sorted(carpetas_count.keys()):
            print(f"   {carpeta}: {carpetas_count[carpeta]} archivos")
        
        return archivos_con_sufijo
    else:
        print(f"\n✅ No se encontraron archivos con sufijos numéricos")
        return []

In [54]:
# Ejecutar la búsqueda
archivos_con_sufijo = encontrar_archivos_con_sufijo_numerico()

🔍 Buscando archivos con sufijos numéricos (_N)...

📊 RESUMEN:
Total de archivos procesados: 259
Archivos con sufijo numérico encontrados: 2

📁 ARCHIVOS CON SUFIJOS NUMÉRICOS:

📂 IWSIM11\Practica_5
   ✓ MARINA_NAVARRO_PAULA_P5_IWSIM11_2.pdf (sufijo: _2)
     → Nombre original sería: MARINA_NAVARRO_PAULA_P5_IWSIM11.pdf

📂 IWSIM12\Practica_5
   ✓ MARINA_NAVARRO_PAULA_P5_IWSIM12_2.pdf (sufijo: _2)
     → Nombre original sería: MARINA_NAVARRO_PAULA_P5_IWSIM12.pdf

📈 ESTADÍSTICAS POR SUFIJO:
   Sufijo _2: 2 archivos

📂 ESTADÍSTICAS POR CARPETA:
   IWSIM11\Practica_5: 1 archivos
   IWSIM12\Practica_5: 1 archivos


In [55]:
import os

# Filtrar archivos que NO contienen 'Paula' en el nombre
archivos_a_eliminar = [archivo for archivo in archivos_con_sufijo if 'PAULA' not in archivo['nombre'].upper()]

print(f"📊 RESUMEN DE ELIMINACIÓN:")
print(f"Total de archivos con sufijo: {len(archivos_con_sufijo)}")
print(f"Archivos que contienen 'Paula' (se conservarán): {len(archivos_con_sufijo) - len(archivos_a_eliminar)}")
print(f"Archivos a eliminar: {len(archivos_a_eliminar)}")

if archivos_a_eliminar:
    print(f"\n🗑️ ELIMINANDO ARCHIVOS:")
    archivos_eliminados = 0
    errores_eliminacion = []
    
    for archivo in archivos_a_eliminar:
        try:
            archivo['ruta_completa'].unlink()  # Eliminar el archivo
            archivos_eliminados += 1
            print(f"  ✅ Eliminado: {archivo['ruta_relativa']}")
        except Exception as e:
            error_msg = f"Error eliminando {archivo['ruta_relativa']}: {e}"
            errores_eliminacion.append(error_msg)
            print(f"  ❌ {error_msg}")
    
    print(f"\n📈 RESULTADO:")
    print(f"Archivos eliminados exitosamente: {archivos_eliminados}")
    
    if errores_eliminacion:
        print(f"Errores durante la eliminación: {len(errores_eliminacion)}")
        for error in errores_eliminacion:
            print(f"  - {error}")
    else:
        print("✅ Todos los archivos se eliminaron sin errores")
        
    # Mostrar archivos conservados (con Paula)
    archivos_conservados = [archivo for archivo in archivos_con_sufijo if 'PAULA' in archivo['nombre'].upper()]
    if archivos_conservados:
        print(f"\n💾 ARCHIVOS CONSERVADOS (contienen 'Paula'):")
        for archivo in archivos_conservados:
            print(f"  ✓ {archivo['ruta_relativa']}")
else:
    print(f"\n✅ No hay archivos para eliminar")

📊 RESUMEN DE ELIMINACIÓN:
Total de archivos con sufijo: 2
Archivos que contienen 'Paula' (se conservarán): 2
Archivos a eliminar: 0

✅ No hay archivos para eliminar


In [56]:
def buscar_alumnos_practicas_2_4(ruta_examenes="../data/examenes_procesados/"):
    """
    Busca alumnos que han entregado prácticas 2 o 4 basándose en la estructura de carpetas
    y nombres de archivos. Marca con * los que están en carpeta problemático.
    """
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"❌ La ruta {ruta_examenes} no existe")
        return []
    
    alumnos_practicas_2_4 = []
    
    print("🔍 Buscando entregas de prácticas 2 y 4...")
    print("=" * 60)
    
    # Recorrer todas las carpetas de grupos
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir():
            continue
            
        # Procesar carpetas de grupos normales
        if carpeta_grupo.name not in ["extraviados", "eliminados"]:
            # Si es problemático, marcar con asterisco
            asterisco = " *" if carpeta_grupo.name == "problemático" else ""
            
            if carpeta_grupo.name == "problemático":
                # En problemático, buscar directamente archivos PDF
                for archivo_pdf in carpeta_grupo.glob("*.pdf"):
                    practica = detectar_practica_del_nombre(archivo_pdf.name)
                    if practica in ['2', '4']:
                        info_alumno = extraer_info_alumno(archivo_pdf, practica, "problemático", True)
                        if info_alumno:
                            alumnos_practicas_2_4.append(info_alumno)
            else:
                # Buscar en subcarpetas de prácticas
                for subcarpeta in carpeta_grupo.iterdir():
                    if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                        practica_num = subcarpeta.name.replace("Practica_", "")
                        
                        if practica_num in ['2', '4']:
                            print(f"📂 Encontrada {subcarpeta.name} en grupo {carpeta_grupo.name}")
                            
                            for archivo_pdf in subcarpeta.glob("*.pdf"):
                                info_alumno = extraer_info_alumno(archivo_pdf, practica_num, carpeta_grupo.name, False)
                                if info_alumno:
                                    alumnos_practicas_2_4.append(info_alumno)
    
    print("\n" + "=" * 60)
    print(f"📊 RESUMEN: Encontrados {len(alumnos_practicas_2_4)} alumnos con prácticas 2 o 4")
    
    if alumnos_practicas_2_4:
        # Organizar por grupo y práctica
        por_grupo_practica = {}
        
        for alumno in alumnos_practicas_2_4:
            grupo = alumno['grupo']
            practica = alumno['practica']
            key = f"{grupo}_P{practica}"
            
            if key not in por_grupo_practica:
                por_grupo_practica[key] = []
            por_grupo_practica[key].append(alumno)
        
        print(f"\n📋 LISTADO POR GRUPO Y PRÁCTICA:")
        print("=" * 60)
        
        for key in sorted(por_grupo_practica.keys()):
            grupo, practica_info = key.split('_P')
            asterisco = " *" if any(a['problematico'] for a in por_grupo_practica[key]) else ""
            
            print(f"\n📚 GRUPO {grupo} - PRÁCTICA {practica_info}{asterisco}")
            print("-" * 40)
            
            for alumno in sorted(por_grupo_practica[key], key=lambda x: (x['apellidos'], x['nombre'])):
                asterisco_individual = " *" if alumno['problematico'] else ""
                print(f"  • {alumno['apellidos']}, {alumno['nombre']}{asterisco_individual}")
    
    return alumnos_practicas_2_4

def extraer_info_alumno(archivo_pdf, practica, grupo, es_problematico):
    """Extrae información del alumno desde el nombre del archivo"""
    nombre_archivo = archivo_pdf.stem
    
    # Limpiar el nombre del archivo para extraer apellidos y nombre
    nombre_limpio = nombre_archivo
    
    # Remover elementos conocidos (práctica, grupo, etc.)
    patrones_a_remover = [
        rf'_P{practica}', rf'P{practica}_', rf'P{practica}$',
        rf'_{grupo}', rf'{grupo}_', rf'^{grupo}',
        r'_CITIM\d+', r'_IWSIM\d+', r'_CITIT\d+', r'_IWSIT\d+',
        r'CITIM\d+_', r'IWSIM\d+_', r'CITIT\d+_', r'IWSIT\d+_',
        r'_\d+$'  # Números al final
    ]
    
    for patron in patrones_a_remover:
        nombre_limpio = re.sub(patron, '', nombre_limpio, flags=re.IGNORECASE)
    
    # Limpiar guiones bajos múltiples
    nombre_limpio = re.sub(r'_+', '_', nombre_limpio).strip('_')
    
    # Separar apellidos y nombre
    partes = nombre_limpio.split('_')
    
    if len(partes) >= 2:
        # Asumir que la última parte es el nombre y el resto apellidos
        apellidos = '_'.join(partes[:-1]).replace('_', ' ')
        nombre = partes[-1]
    elif len(partes) == 1:
        apellidos = partes[0]
        nombre = ''
    else:
        apellidos = nombre_archivo
        nombre = ''
    
    return {
        'archivo': archivo_pdf.name,
        'apellidos': apellidos,
        'nombre': nombre,
        'practica': practica,
        'grupo': grupo,
        'problematico': es_problematico
    }

def detectar_practica_del_nombre(nombre_archivo):
    """Detecta el número de práctica del nombre del archivo"""
    nombre = nombre_archivo.upper()
    
    # Buscar patrones de práctica
    patrones = [
        r'P(\d)',
        r'_(\d)_',
        r'PRACTICA_?(\d)',
    ]
    
    for patron in patrones:
        match = re.search(patron, nombre)
        if match:
            return match.group(1)
    
    return None

In [ ]:
# Buscar a Hernandez Garnacho Jose Angel en el DataFrame
#busqueda = df_con_practicas_y_examenes[
#    (df_con_practicas_y_examenes['Apellido(s)'].str.contains('FUERIS FRUTOS', case=False, na=False)) &
#    (df_con_practicas_y_examenes['Nombre'].str.contains('MANUEL', case=False, na=False))
#]
#print("Resultados de búsqueda para Hernandez Garnacho Jose Angel:")
#print(busqueda[['Nombre', 'Apellido(s)', 'Grupos']])

In [57]:
#alumnos_con_practicas_2_4 = buscar_alumnos_practicas_2_4()

In [58]:
def extraer_info_alumno_citit(archivo_pdf, practica, grupo, es_problematico):
    """
    Extrae información del alumno desde el nombre del archivo para grupos CITIT/IWSIT
    """
    nombre_archivo = archivo_pdf.stem
    
    # Limpiar el nombre del archivo para extraer apellidos y nombre
    nombre_limpio = nombre_archivo
    
    # Remover elementos conocidos (práctica, grupo, etc.)
    patrones_a_remover = [
        rf'_P{practica}', rf'P{practica}_', rf'P{practica}$',
        rf'_{grupo}', rf'{grupo}_', rf'^{grupo}',
        r'_CITIT\d+', r'_IWSIT\d+', r'_CITIM\d+', r'_IWSIM\d+',
        r'CITIT\d+_', r'IWSIT\d+_', r'CITIM\d+_', r'IWSIM\d+_',
        r'_\d+$'  # Números al final
    ]
    
    for patron in patrones_a_remover:
        nombre_limpio = re.sub(patron, '', nombre_limpio, flags=re.IGNORECASE)
    
    # Limpiar guiones bajos múltiples
    nombre_limpio = re.sub(r'_+', '_', nombre_limpio).strip('_')
    
    # Separar apellidos y nombre
    partes = nombre_limpio.split('_')
    
    if len(partes) >= 2:
        # Asumir que la última parte es el nombre y el resto apellidos
        apellidos = '_'.join(partes[:-1]).replace('_', ' ')
        nombre = partes[-1]
    elif len(partes) == 1:
        apellidos = partes[0]
        nombre = ''
    else:
        apellidos = nombre_archivo
        nombre = ''
    
    return {
        'archivo': archivo_pdf.name,
        'apellidos': apellidos,
        'nombre': nombre,
        'practica': practica,
        'grupo': grupo,
        'problematico': es_problematico
    }

def buscar_alumnos_grupos_citit_iwsit(ruta_examenes="../data/examenes_procesados/"):
    """
    Busca alumnos de los grupos CITIT11, CITIT12, IWSIT11, IWSIT12 clasificados por práctica.
    Verifica el grupo correcto consultando el DataFrame de alumnos.
    """
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"La ruta {ruta_examenes} no existe")
        return []
    
    # Grupos objetivo
    grupos_objetivo = ['CITIT11', 'CITIT12', 'IWSIT11', 'IWSIT12']
    alumnos_encontrados = []
    
    print("Buscando alumnos de grupos CITIT/IWSIT...")
    print("=" * 60)
    
    # Recorrer todas las carpetas de grupos
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir():
            continue
            
        # Verificar si es uno de los grupos objetivo O carpeta problemático
        es_grupo_objetivo = carpeta_grupo.name in grupos_objetivo
        es_problematico = carpeta_grupo.name == "problemático"
        
        if es_grupo_objetivo or es_problematico:
            print(f"\nProcesando carpeta: {carpeta_grupo.name}")
            
            if es_problematico:
                # En problemático, buscar directamente archivos PDF y verificar si son de grupos objetivo
                for archivo_pdf in carpeta_grupo.glob("*.pdf"):
                    grupo_detectado = detectar_grupo_del_nombre(archivo_pdf.name)
                    practica = detectar_practica_del_nombre(archivo_pdf.name)
                    if practica:
                        info_alumno = extraer_info_alumno_citit(archivo_pdf, practica, grupo_detectado or carpeta_grupo.name, True)
                        if info_alumno:
                            # Verificar grupo real en el DataFrame
                            grupo_real = verificar_grupo_en_dataframe(info_alumno['apellidos'], info_alumno['nombre'])
                            if grupo_real in grupos_objetivo:
                                info_alumno['grupo_real'] = grupo_real
                                info_alumno['grupo_carpeta'] = carpeta_grupo.name
                                alumnos_encontrados.append(info_alumno)
                                print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']} (P{practica}, {grupo_real}) *")
            else:
                # Buscar en subcarpetas de prácticas
                for subcarpeta in carpeta_grupo.iterdir():
                    if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                        practica_num = subcarpeta.name.replace("Practica_", "")
                        print(f"  {subcarpeta.name}")
                        
                        for archivo_pdf in subcarpeta.glob("*.pdf"):
                            info_alumno = extraer_info_alumno_citit(archivo_pdf, practica_num, carpeta_grupo.name, False)
                            if info_alumno:
                                # Verificar grupo real en el DataFrame
                                grupo_real = verificar_grupo_en_dataframe(info_alumno['apellidos'], info_alumno['nombre'])
                                if grupo_real in grupos_objetivo:
                                    info_alumno['grupo_real'] = grupo_real
                                    info_alumno['grupo_carpeta'] = carpeta_grupo.name
                                    alumnos_encontrados.append(info_alumno)
                                    
                                    # Mostrar advertencia si el grupo de carpeta no coincide con el real
                                    if grupo_real != carpeta_grupo.name:
                                        print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']} (Grupo real: {grupo_real}, Carpeta: {carpeta_grupo.name}) ⚠️")
                                    else:
                                        print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']}")
    
    print("\n" + "=" * 60)
    print(f"RESUMEN: Encontrados {len(alumnos_encontrados)} alumnos de grupos CITIT/IWSIT")
    
    if alumnos_encontrados:
        # Organizar por práctica y luego por grupo REAL
        por_practica = {}
        
        for alumno in alumnos_encontrados:
            practica = alumno['practica']
            if practica not in por_practica:
                por_practica[practica] = {}
            
            grupo = alumno['grupo_real']  # Usar grupo real del DataFrame
            if grupo not in por_practica[practica]:
                por_practica[practica][grupo] = []
            
            por_practica[practica][grupo].append(alumno)
        
        print(f"\nLISTADO SEPARADO EN DOS LISTAS:")
        print("=" * 80)
        
        # Ordenar por número de práctica
        for practica in sorted(por_practica.keys(), key=lambda x: int(x) if x.isdigit() else 999):
            print(f"\nPractica {practica}:")
            print("-" * 50)
            
            # Ordenar grupos alfabéticamente
            for grupo in sorted(por_practica[practica].keys()):
                alumnos_grupo = por_practica[practica][grupo]
                
                # Separar en problemáticos y normales
                alumnos_normales = [alumno for alumno in alumnos_grupo if not alumno['problematico']]
                alumnos_problematicos = [alumno for alumno in alumnos_grupo if alumno['problematico']]
                
                print(f"\nGrupo {grupo}:")
                
                # Lista de alumnos normales
                if alumnos_normales:
                    print("  Alumnos normales:")
                    for alumno in sorted(alumnos_normales, key=lambda x: (x['apellidos'], x['nombre'])):
                        print(f"    {alumno['apellidos']}, {alumno['nombre']}")
                
                # Lista de alumnos problemáticos (dentro del mismo grupo)
                if alumnos_problematicos:
                    print("  Alumnos problemáticos:")
                    for alumno in sorted(alumnos_problematicos, key=lambda x: (x['apellidos'], x['nombre'])):
                        print(f"    {alumno['apellidos']}, {alumno['nombre']} *")
        
        print(f"\n" + "-" * 40)
        print(f"RESUMEN POR GRUPO:")
        print("-" * 40)
        
        resumen_grupos = {}
        for alumno in alumnos_encontrados:
            grupo = alumno['grupo_real']  # Usar grupo real
            if grupo not in resumen_grupos:
                resumen_grupos[grupo] = {'total': 0, 'problematicos': 0}
            resumen_grupos[grupo]['total'] += 1
            if alumno['problematico']:
                resumen_grupos[grupo]['problematicos'] += 1
        
        for grupo in sorted(resumen_grupos.keys()):
            total = resumen_grupos[grupo]['total']
            problematicos = resumen_grupos[grupo]['problematicos']
            print(f"  {grupo}: {total} alumnos (problemáticos: {problematicos})")
    
    return alumnos_encontrados

def verificar_grupo_en_dataframe(apellidos, nombre):
    """
    Verifica el grupo real del alumno consultando el DataFrame df_con_practicas_y_examenes
    """
    if 'df_con_practicas_y_examenes' not in globals():
        print("⚠️ DataFrame df_con_practicas_y_examenes no está disponible")
        return "GRUPO_NO_ENCONTRADO"
    
    apellidos_limpio = apellidos.upper().strip()
    nombre_limpio = nombre.upper().strip()
    
    # Buscar coincidencia exacta primero
    mask_exacta = (df_con_practicas_y_examenes['Apellido(s)'].str.upper().str.strip() == apellidos_limpio) & \
                  (df_con_practicas_y_examenes['Nombre'].str.upper().str.strip() == nombre_limpio)
    
    if mask_exacta.any():
        return df_con_practicas_y_examenes.loc[mask_exacta, 'Grupos'].iloc[0]
    
    # Si no hay coincidencia exacta, buscar coincidencia parcial
    if apellidos_limpio and nombre_limpio:
        mask_parcial = df_con_practicas_y_examenes['Apellido(s)'].str.upper().str.contains(apellidos_limpio[:5], na=False) & \
                       df_con_practicas_y_examenes['Nombre'].str.upper().str.contains(nombre_limpio[:3], na=False)
        
        if mask_parcial.any():
            return df_con_practicas_y_examenes.loc[mask_parcial, 'Grupos'].iloc[0]
    
    return "GRUPO_NO_ENCONTRADO"

In [ ]:
def buscar_alumnos_grupos_citit_iwsit(ruta_examenes="../data/examenes_procesados/"):
    """
    Busca alumnos de los grupos CITIT11, CITIT12, IWSIT11, IWSIT12 clasificados por práctica.
    Verifica el grupo correcto consultando el DataFrame de alumnos.
    """
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"La ruta {ruta_examenes} no existe")
        return []
    
    # Grupos objetivo
    grupos_objetivo = ['CITIT11', 'CITIT12', 'IWSIT11', 'IWSIT12']
    alumnos_encontrados = []
    
    print("Buscando alumnos de grupos CITIT/IWSIT...")
    print("=" * 60)
    
    # Recorrer todas las carpetas de grupos
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir():
            continue
            
        # Verificar si es uno de los grupos objetivo O carpeta problemático
        es_grupo_objetivo = carpeta_grupo.name in grupos_objetivo
        es_problematico = carpeta_grupo.name == "problemático"
        
        if es_grupo_objetivo or es_problematico:
            print(f"\nProcesando carpeta: {carpeta_grupo.name}")
            
            if es_problematico:
                # En problemático, buscar directamente archivos PDF y verificar si son de grupos objetivo
                for archivo_pdf in carpeta_grupo.glob("*.pdf"):
                    grupo_detectado = detectar_grupo_del_nombre(archivo_pdf.name)
                    practica = detectar_practica_del_nombre(archivo_pdf.name)
                    if practica:
                        info_alumno = extraer_info_alumno_citit(archivo_pdf, practica, grupo_detectado or carpeta_grupo.name, True)
                        if info_alumno:
                            # Verificar grupo real en el DataFrame
                            grupo_real = verificar_grupo_en_dataframe(info_alumno['apellidos'], info_alumno['nombre'])
                            if grupo_real in grupos_objetivo:
                                info_alumno['grupo_real'] = grupo_real
                                info_alumno['grupo_carpeta'] = carpeta_grupo.name
                                alumnos_encontrados.append(info_alumno)
                                print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']} (P{practica}, {grupo_real}) *")
            else:
                # Buscar en subcarpetas de prácticas
                for subcarpeta in carpeta_grupo.iterdir():
                    if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                        practica_num = subcarpeta.name.replace("Practica_", "")
                        print(f"  {subcarpeta.name}")
                        
                        for archivo_pdf in subcarpeta.glob("*.pdf"):
                            info_alumno = extraer_info_alumno_citit(archivo_pdf, practica_num, carpeta_grupo.name, False)
                            if info_alumno:
                                # Verificar grupo real en el DataFrame
                                grupo_real = verificar_grupo_en_dataframe(info_alumno['apellidos'], info_alumno['nombre'])
                                if grupo_real in grupos_objetivo:
                                    info_alumno['grupo_real'] = grupo_real
                                    info_alumno['grupo_carpeta'] = carpeta_grupo.name
                                    alumnos_encontrados.append(info_alumno)
                                    
                                    # Mostrar advertencia si el grupo de carpeta no coincide con el real
                                    if grupo_real != carpeta_grupo.name:
                                        print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']} (Grupo real: {grupo_real}, Carpeta: {carpeta_grupo.name}) ⚠️")
                                    else:
                                        print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']}")
    
    print("\n" + "=" * 60)
    print(f"RESUMEN: Encontrados {len(alumnos_encontrados)} alumnos de grupos CITIT/IWSIT")
    
    if alumnos_encontrados:
        # Organizar por práctica y luego por grupo REAL
        por_practica = {}
        
        for alumno in alumnos_encontrados:
            practica = alumno['practica']
            if practica not in por_practica:
                por_practica[practica] = {}
            
            grupo = alumno['grupo_real']  # Usar grupo real del DataFrame
            if grupo not in por_practica[practica]:
                por_practica[practica][grupo] = []
            
            por_practica[practica][grupo].append(alumno)
        
        print(f"\nLISTADO SEPARADO EN DOS LISTAS:")
        print("=" * 80)
        
        # Ordenar por número de práctica
        for practica in sorted(por_practica.keys(), key=lambda x: int(x) if x.isdigit() else 999):
            print(f"\nPractica {practica}:")
            print("-" * 50)
            
            # Ordenar grupos alfabéticamente
            for grupo in sorted(por_practica[practica].keys()):
                alumnos_grupo = por_practica[practica][grupo]
                
                # Separar en problemáticos y normales
                alumnos_normales = [alumno for alumno in alumnos_grupo if not alumno['problematico']]
                alumnos_problematicos = [alumno for alumno in alumnos_grupo if alumno['problematico']]
                
                print(f"\nGrupo {grupo}:")
                
                # Lista de alumnos normales
                if alumnos_normales:
                    print("  Alumnos normales:")
                    for alumno in sorted(alumnos_normales, key=lambda x: (x['apellidos'], x['nombre'])):
                        print(f"    {alumno['apellidos']}, {alumno['nombre']}")
                
                # Lista de alumnos problemáticos (dentro del mismo grupo)
                if alumnos_problematicos:
                    print("  Alumnos problemáticos:")
                    for alumno in sorted(alumnos_problematicos, key=lambda x: (x['apellidos'], x['nombre'])):
                        print(f"    {alumno['apellidos']}, {alumno['nombre']} *")
        
        print(f"\n" + "-" * 40)
        print(f"RESUMEN POR GRUPO:")
        print("-" * 40)
        
        resumen_grupos = {}
        for alumno in alumnos_encontrados:
            grupo = alumno['grupo_real']  # Usar grupo real
            if grupo not in resumen_grupos:
                resumen_grupos[grupo] = {'total': 0, 'problematicos': 0}
            resumen_grupos[grupo]['total'] += 1
            if alumno['problematico']:
                resumen_grupos[grupo]['problematicos'] += 1
        
        for grupo in sorted(resumen_grupos.keys()):
            total = resumen_grupos[grupo]['total']
            problematicos = resumen_grupos[grupo]['problematicos']
            print(f"  {grupo}: {total} alumnos (problemáticos: {problematicos})")
    
    return alumnos_encontrados

def verificar_grupo_en_dataframe(apellidos, nombre):
    """
    Verifica el grupo real del alumno consultando el DataFrame df_con_practicas_y_examenes
    """
    if 'df_con_practicas_y_examenes' not in globals():
        print("⚠️ DataFrame df_con_practicas_y_examenes no está disponible")
        return "GRUPO_NO_ENCONTRADO"
    
    apellidos_limpio = apellidos.upper().strip()
    nombre_limpio = nombre.upper().strip()
    
    # Buscar coincidencia exacta primero
    mask_exacta = (df_con_practicas_y_examenes['Apellido(s)'].str.upper().str.strip() == apellidos_limpio) & \
                  (df_con_practicas_y_examenes['Nombre'].str.upper().str.strip() == nombre_limpio)
    
    if mask_exacta.any():
        return df_con_practicas_y_examenes.loc[mask_exacta, 'Grupos'].iloc[0]
    
    # Si no hay coincidencia exacta, buscar coincidencia parcial
    if apellidos_limpio and nombre_limpio:
        mask_parcial = df_con_practicas_y_examenes['Apellido(s)'].str.upper().str.contains(apellidos_limpio[:5], na=False) & \
                       df_con_practicas_y_examenes['Nombre'].str.upper().str.contains(nombre_limpio[:3], na=False)
        
        if mask_parcial.any():
            return df_con_practicas_y_examenes.loc[mask_parcial, 'Grupos'].iloc[0]
    
    return "GRUPO_NO_ENCONTRADO"

In [59]:

# Ejecutar la búsqueda
#alumnos_citit_iwsit = buscar_alumnos_grupos_citit_iwsit()

In [60]:
import os
from pathlib import Path
import re
from pypdf import PdfReader
import hashlib

def buscar_origen_lote(nombre_archivo_objetivo, ruta_lotes="../data/raw/", ruta_examenes="../data/examenes_procesados/"):
    """
    Busca de qué lote proviene un archivo específico comparando contenido de PDFs
    """
    
    # Buscar el archivo objetivo en examenes_procesados
    archivo_objetivo = None
    ruta_examenes = Path(ruta_examenes)
    
    for root, dirs, files in os.walk(ruta_examenes):
        for file in files:
            if nombre_archivo_objetivo.lower() in file.lower():
                archivo_objetivo = Path(root) / file
                break
        if archivo_objetivo:
            break
    
    if not archivo_objetivo or not archivo_objetivo.exists():
        print(f"❌ No se encontró el archivo {nombre_archivo_objetivo} en examenes_procesados")
        return None
    
    print(f"🎯 Archivo encontrado: {archivo_objetivo}")
    
    # Leer el contenido del archivo objetivo para comparar
    try:
        reader_objetivo = PdfReader(archivo_objetivo)
        contenido_objetivo = ""
        for page in reader_objetivo.pages:
            contenido_objetivo += page.extract_text()
        
        # Hash del contenido para comparación rápida
        hash_objetivo = hashlib.md5(contenido_objetivo.encode()).hexdigest()
        print(f"📝 Hash del archivo objetivo: {hash_objetivo[:16]}...")
        
    except Exception as e:
        print(f"❌ Error leyendo archivo objetivo: {e}")
        return None
    
    # Buscar en todos los lotes
    ruta_lotes = Path(ruta_lotes)
    if not ruta_lotes.exists():
        print(f"❌ La ruta de lotes {ruta_lotes} no existe")
        return None
    
    print(f"\n🔍 Buscando en lotes de {ruta_lotes}...")
    
    lotes_encontrados = []
    
    for archivo_lote in ruta_lotes.glob("*.pdf"):
        print(f"  📄 Revisando {archivo_lote.name}...")
        
        try:
            reader_lote = PdfReader(archivo_lote)
            num_paginas = len(reader_lote.pages)
            
            # Revisar cada página del lote
            for i in range(num_paginas):
                try:
                    page = reader_lote.pages[i]
                    contenido_pagina = page.extract_text()
                    hash_pagina = hashlib.md5(contenido_pagina.encode()).hexdigest()
                    
                    # Comparar hashes
                    if hash_pagina == hash_objetivo:
                        lotes_encontrados.append({
                            'lote': archivo_lote.name,
                            'pagina': i + 1,
                            'total_paginas': num_paginas,
                            'coincidencia': 'exacta'
                        })
                        print(f"    ✅ COINCIDENCIA EXACTA: Página {i + 1} de {num_paginas}")
                    
                    # También buscar coincidencias parciales por texto clave
                    elif buscar_coincidencias_texto(contenido_pagina, contenido_objetivo):
                        lotes_encontrados.append({
                            'lote': archivo_lote.name,
                            'pagina': i + 1,
                            'total_paginas': num_paginas,
                            'coincidencia': 'parcial'
                        })
                        print(f"    🔍 Coincidencia parcial: Página {i + 1} de {num_paginas}")
                        
                except Exception as e:
                    print(f"    ⚠️ Error leyendo página {i + 1}: {e}")
                    continue
                    
        except Exception as e:
            print(f"    ❌ Error leyendo lote {archivo_lote.name}: {e}")
            continue
    
    # Mostrar resultados
    print(f"\n" + "="*60)
    print(f"📊 RESULTADOS PARA: {nombre_archivo_objetivo}")
    print(f"="*60)
    
    if lotes_encontrados:
        print(f"🎉 Encontradas {len(lotes_encontrados)} coincidencias:")
        
        for coincidencia in lotes_encontrados:
            tipo_icono = "🎯" if coincidencia['coincidencia'] == 'exacta' else "🔍"
            print(f"\n{tipo_icono} LOTE: {coincidencia['lote']}")
            print(f"   📍 Página: {coincidencia['pagina']} de {coincidencia['total_paginas']}")
            print(f"   📝 Tipo: {coincidencia['coincidencia']}")
            
            # Calcular posición aproximada en el examen
            if coincidencia['pagina'] % 2 == 1:  # Página impar
                examen_num = (coincidencia['pagina'] + 1) // 2
                print(f"   📄 Probablemente examen #{examen_num} (página 1)")
            else:  # Página par
                examen_num = coincidencia['pagina'] // 2
                print(f"   📄 Probablemente examen #{examen_num} (página 2)")
                
        return lotes_encontrados
    else:
        print("❌ No se encontraron coincidencias en ningún lote")
        print("\n💡 Posibles causas:")
        print("   - El archivo fue modificado después de la extracción")
        print("   - El archivo proviene de una fuente diferente")
        print("   - Error en la comparación de contenido")
        return None

def buscar_coincidencias_texto(texto1, texto2):
    """Busca coincidencias parciales entre dos textos"""
    # Limpiar y normalizar textos
    texto1_limpio = re.sub(r'\s+', ' ', texto1.upper().strip())
    texto2_limpio = re.sub(r'\s+', ' ', texto2.upper().strip())
    
    # Buscar fragmentos comunes significativos
    palabras1 = set(texto1_limpio.split())
    palabras2 = set(texto2_limpio.split())
    
    # Calcular similitud (Jaccard)
    interseccion = len(palabras1.intersection(palabras2))
    union = len(palabras1.union(palabras2))
    
    if union == 0:
        return False
    
    similitud = interseccion / union
    return similitud > 0.7  # 70% de similitud

In [86]:
import numpy as np
from PIL import Image, ImageChops
from tqdm import tqdm
import os
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader

def comparar_pdfs_visuales_con_tqdm(nombre_archivo_objetivo, ruta_saved="../data/saved/", ruta_lotes="../data/raw/", 
                                   mostrar_detalles=False, parar_en_100=False):
    """
    Compara un archivo objetivo con los archivos en /saved y los lotes originales
    usando comparación visual y número de páginas con tqdm y mostrando TOP 3 similitudes.
    
    Args:
        mostrar_detalles: Si True, muestra prints detallados. Si False, solo muestra tqdm y resultados.
        parar_en_100: Si True, para la búsqueda al encontrar 100% similitud.
    """
    
    # Buscar el archivo objetivo en examenes_procesados
    archivo_objetivo = None
    ruta_examenes = Path("../data/examenes_procesados/")
    
    for root, dirs, files in os.walk(ruta_examenes):
        for file in files:
            if nombre_archivo_objetivo.lower() in file.lower():
                archivo_objetivo = Path(root) / file
                break
        if archivo_objetivo:
            break
    
    if not archivo_objetivo or not archivo_objetivo.exists():
        print(f"❌ No se encontró el archivo {nombre_archivo_objetivo} en examenes_procesados")
        return None
    
    print(f"🎯 Archivo encontrado: {archivo_objetivo}")
    
    # Obtener número de páginas del objetivo
    try:
        reader_objetivo = PdfReader(archivo_objetivo)
        num_paginas_objetivo = len(reader_objetivo.pages)
        print(f"📄 Páginas del archivo objetivo: {num_paginas_objetivo}")
    except Exception as e:
        print(f"❌ Error leyendo archivo objetivo: {e}")
        return None
    
    # Convertir páginas del objetivo a imágenes para comparación
    print(f"🖼️ Convirtiendo archivo objetivo a imágenes...")
    try:
        imagenes_objetivo = convert_from_path(
            archivo_objetivo, 
            dpi=100,
            fmt='jpeg'
        )
    except Exception as e:
        print(f"❌ Error convirtiendo archivo objetivo: {e}")
        return None
    
    # Listas para almacenar similitudes y TOP 3 actuales
    todas_las_similitudes = []
    todas_las_similitudes_lotes = []
    coincidencias_encontradas = []
    parada_activada = False
    
    def mostrar_top3_actual():
        """Muestra los TOP 3 actuales combinando saved y lotes"""
        # Combinar todas las similitudes válidas
        similitudes_combinadas = []
        
        # Añadir de saved
        for s in todas_las_similitudes:
            if s['similitud'] > 0:
                similitudes_combinadas.append(s)
        
        # Añadir de lotes
        for s in todas_las_similitudes_lotes:
            if s['similitud'] > 0:
                similitudes_combinadas.append(s)
        
        if similitudes_combinadas:
            top_3 = sorted(similitudes_combinadas, key=lambda x: x['similitud'], reverse=True)[:3]
            
            top_str = "TOP 3: "
            for i, sim in enumerate(top_3, 1):
                icono = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
                if sim['tipo'] == 'lote':
                    top_str += f"{icono}{sim['similitud']:.3f}({sim['archivo'][:15]}..Ex#{sim['examen_num']}) "
                else:
                    top_str += f"{icono}{sim['similitud']:.3f}({sim['archivo'][:15]}...) "
            
            return top_str
        else:
            return "TOP 3: Sin coincidencias aún"
    
    # 1. COMPARAR CON ARCHIVOS EN /saved
    print(f"\n🔍 Comparando con archivos en {ruta_saved}...")
    ruta_saved = Path(ruta_saved)
    
    if ruta_saved.exists():
        archivos_saved = list(ruta_saved.glob("*.pdf"))
        print(f"📁 Encontrados {len(archivos_saved)} archivos en saved")
        
        # Usar tqdm para mostrar progreso CON TOP 3 ACTUALIZADO
        progress_bar = tqdm(archivos_saved, desc="💾 Comparando con /saved", unit="archivo")
        
        for archivo_saved in progress_bar:
            if parada_activada:
                break
                
            try:
                reader_saved = PdfReader(archivo_saved)
                num_paginas_saved = len(reader_saved.pages)
                
                # Solo comparar si tienen el mismo número de páginas
                if num_paginas_saved == num_paginas_objetivo:
                    if mostrar_detalles:
                        tqdm.write(f"  📄 {archivo_saved.name} ({num_paginas_saved} páginas) - Comparando...")
                    
                    # Convertir a imágenes
                    imagenes_saved = convert_from_path(archivo_saved, dpi=100, fmt='jpeg')
                    
                    # Comparar cada página
                    similitud_total = comparar_imagenes_paginas(imagenes_objetivo, imagenes_saved)
                    
                    # GUARDAR SIMILITUD
                    similitud_info = {
                        'archivo': archivo_saved.name,
                        'ruta': str(archivo_saved),
                        'tipo': 'saved',
                        'similitud': similitud_total,
                        'paginas': num_paginas_saved
                    }
                    todas_las_similitudes.append(similitud_info)
                    
                    if similitud_total > 0.85:  # 85% de similitud
                        coincidencias_encontradas.append(similitud_info)
                        if mostrar_detalles:
                            tqdm.write(f"    ✅ COINCIDENCIA: {similitud_total:.2%}")
                        
                        # PARADA AUTOMÁTICA al 100%
                        if parar_en_100 and similitud_total >= 0.999:  # 99.9% se considera 100%
                            tqdm.write(f"    🎯 ¡100% SIMILITUD ENCONTRADA! Deteniendo búsqueda...")
                            parada_activada = True
                            break
                    elif mostrar_detalles:
                        tqdm.write(f"    ❌ No coincide: {similitud_total:.2%}")
                else:
                    # Añadir con similitud 0 para estadísticas
                    todas_las_similitudes.append({
                        'archivo': archivo_saved.name,
                        'ruta': str(archivo_saved),
                        'tipo': 'saved',
                        'similitud': 0.0,
                        'paginas': f"{num_paginas_saved} páginas"
                    })
                
                # ACTUALIZAR DESCRIPCIÓN CON TOP 3 ACTUAL
                progress_bar.set_description(f"💾 Saved | {mostrar_top3_actual()}")
                    
            except Exception as e:
                if mostrar_detalles:
                    tqdm.write(f"  ❌ Error procesando {archivo_saved.name}: {e}")
                continue
        
        progress_bar.close()
    
    # 2. COMPARAR CON LOTES ORIGINALES (solo si no se activó la parada)
    if not parada_activada:
        print(f"\n🔍 Comparando con lotes originales en {ruta_lotes}...")
        ruta_lotes = Path(ruta_lotes)
        
        if ruta_lotes.exists():
            archivos_lotes = list(ruta_lotes.glob("*.pdf"))
            
            progress_bar_lotes = tqdm(archivos_lotes, desc="📦 Comparando con lotes", unit="lote")
            
            for archivo_lote in progress_bar_lotes:
                if parada_activada:
                    break
                    
                if mostrar_detalles:
                    tqdm.write(f"  📄 Revisando {archivo_lote.name}...")
                
                try:
                    reader_lote = PdfReader(archivo_lote)
                    num_paginas_lote = len(reader_lote.pages)
                    
                    # Buscar secuencias de páginas que coincidan en número
                    for inicio in range(0, num_paginas_lote, 2):  # Examenes de 2 páginas
                        if parada_activada:
                            break
                            
                        fin = min(inicio + num_paginas_objetivo, num_paginas_lote)
                        
                        if fin - inicio == num_paginas_objetivo:
                            if mostrar_detalles:
                                tqdm.write(f"    🔍 Comparando páginas {inicio+1}-{fin} del lote...")
                            
                            try:
                                imagenes_lote = convert_from_path(
                                    archivo_lote,
                                    first_page=inicio + 1,
                                    last_page=fin,
                                    dpi=100,
                                    fmt='jpeg'
                                )
                                
                                # Comparar
                                similitud_total = comparar_imagenes_paginas(imagenes_objetivo, imagenes_lote)
                                examen_num = (inicio // 2) + 1
                                
                                # GUARDAR SIMILITUD
                                similitud_info = {
                                    'archivo': archivo_lote.name,
                                    'ruta': str(archivo_lote),
                                    'tipo': 'lote',
                                    'similitud': similitud_total,
                                    'paginas': f"{inicio+1}-{fin}",
                                    'examen_num': examen_num
                                }
                                todas_las_similitudes_lotes.append(similitud_info)
                                
                                if similitud_total > 0.85:
                                    coincidencias_encontradas.append(similitud_info)
                                    if mostrar_detalles:
                                        tqdm.write(f"      ✅ COINCIDENCIA: {similitud_total:.2%} (Examen #{examen_num})")
                                    
                                    # PARADA AUTOMÁTICA al 100%
                                    if parar_en_100 and similitud_total >= 0.999:
                                        tqdm.write(f"      🎯 ¡100% SIMILITUD ENCONTRADA! Deteniendo búsqueda...")
                                        parada_activada = True
                                        break
                                
                            except Exception as e:
                                if mostrar_detalles:
                                    tqdm.write(f"      ❌ Error comparando páginas {inicio+1}-{fin}: {e}")
                                continue
                
                    # ACTUALIZAR DESCRIPCIÓN CON TOP 3 ACTUAL
                    progress_bar_lotes.set_description(f"📦 Lotes | {mostrar_top3_actual()}")
                                
                except Exception as e:
                    if mostrar_detalles:
                        tqdm.write(f"  ❌ Error procesando lote {archivo_lote.name}: {e}")
                    continue
            
            progress_bar_lotes.close()
    else:
        print("\n⏹️ Búsqueda en lotes omitida (parada automática activada)")
    
    # MOSTRAR RESULTADOS FINALES
    print(f"\n" + "="*80)
    print(f"📊 RESULTADOS FINALES PARA: {nombre_archivo_objetivo}")
    print(f"="*80)
    
    if coincidencias_encontradas:
        # Ordenar por similitud descendente
        coincidencias_encontradas.sort(key=lambda x: x['similitud'], reverse=True)
        
        print(f"🎉 Encontradas {len(coincidencias_encontradas)} coincidencias que superan el umbral:")
        
        for i, coincidencia in enumerate(coincidencias_encontradas, 1):
            print(f"\n{i}. 📁 ARCHIVO: {coincidencia['archivo']}")
            print(f"   📍 Ubicación: {coincidencia['tipo']}")
            print(f"   📊 Similitud: {coincidencia['similitud']:.2%}")
            
            if coincidencia['tipo'] == 'lote':
                print(f"   📄 Páginas: {coincidencia['paginas']}")
                print(f"   🔢 Examen #: {coincidencia['examen_num']}")
            else:
                print(f"   📄 Páginas: {coincidencia['paginas']}")
            
            print(f"   🔗 Ruta: {coincidencia['ruta']}")
        
        return coincidencias_encontradas
    else:
        print("❌ No se encontraron coincidencias que superen el umbral (85%)")
        return None

def comparar_imagenes_paginas(imagenes1, imagenes2):
    """
    Compara dos listas de imágenes página por página
    Retorna un porcentaje de similitud promedio
    """
    if len(imagenes1) != len(imagenes2):
        return 0.0
    
    similitudes = []
    
    for img1, img2 in zip(imagenes1, imagenes2):
        # Redimensionar a mismo tamaño si es necesario
        if img1.size != img2.size:
            # Usar el tamaño más pequeño
            nuevo_tamaño = (
                min(img1.size[0], img2.size[0]),
                min(img1.size[1], img2.size[1])
            )
            img1 = img1.resize(nuevo_tamaño)
            img2 = img2.resize(nuevo_tamaño)
        
        # Comparar usando diferencia de píxeles
        diferencia = ImageChops.difference(img1, img2)
        
        # Convertir a array numpy
        arr_diff = np.array(diferencia)
        
        # Calcular similitud (invertir la diferencia)
        diferencia_promedio = np.mean(arr_diff) / 255.0
        similitud = 1.0 - diferencia_promedio
        
        similitudes.append(similitud)
    
    return np.mean(similitudes)

In [88]:
#path_a_comparar = "poner aqui pdf 1 pagina"
#resultado = comparar_pdfs_visuales_con_tqdm(path_a_comparar, parar_en_100=True)

In [61]:
import pandas as pd

# Crear DataFrame con los resultados ordenado por similitud de mayor a menor
df_resultado = pd.DataFrame(resultado)
df_resultado_ordenado = df_resultado.sort_values('similitud', ascending=False)

# Mostrar el DataFrame
#display(df_resultado_ordenado)

NameError: name 'resultado' is not defined

In [91]:
def analizar_examenes_problematicos(ruta_examenes="../data/examenes_procesados/", 
                                   ruta_saved="../data/saved/", 
                                   parar_en_100=False,
                                   umbral_similitud=0.85):
    """
    Analiza todos los exámenes en la carpeta 'problemático' comparando SOLO con saved.
    Muestra los 3 mejores resultados por alumno en tiempo real.
    USA DIRECTAMENTE comparar_pdfs_visuales_con_tqdm - versión simplificada
    """
    
    ruta_problematico = Path(ruta_examenes) / "problemático"
    
    if not ruta_problematico.exists():
        print("❌ No existe la carpeta 'problemático'")
        return pd.DataFrame()
    
    archivos_problematicos = list(ruta_problematico.glob("*.pdf"))
    
    if not archivos_problematicos:
        print("ℹ️ No hay archivos en la carpeta 'problemático'")
        return pd.DataFrame()
    
    print(f"🔍 Analizando {len(archivos_problematicos)} archivos problemáticos...")
    print("💾 Solo comparando con archivos en /saved")
    if parar_en_100:
        print("⏹️ Modo parada automática activado (se detiene al encontrar 100% similitud)")
    
    resultados = []
    
    # Barra de progreso principal con tqdm
    for archivo_objetivo in tqdm(archivos_problematicos, desc="📄 Procesando archivos", unit="archivo"):
        try:
            # Mostrar archivo actual
            tqdm.write(f"\n📄 Procesando: {archivo_objetivo.name}")
            
            # USAR DIRECTAMENTE comparar_pdfs_visuales_con_tqdm con ruta_lotes vacía
            # para que SOLO compare con /saved
            coincidencias = comparar_pdfs_visuales_con_tqdm_solo_saved(
                archivo_objetivo.name,  # Solo el nombre del archivo
                ruta_saved=ruta_saved,
                ruta_examenes=str(ruta_problematico),  # Buscar en problemático
                mostrar_detalles=False,  # Silencioso para no saturar output
                parar_en_100=parar_en_100,
                umbral_similitud=umbral_similitud
            )
            
            if coincidencias:
                # Filtrar solo coincidencias de tipo 'saved' y ordenar por similitud
                coincidencias_saved = [c for c in coincidencias if c.get('tipo') == 'saved']
                mejores_coincidencias = sorted(coincidencias_saved, key=lambda x: x['similitud'], reverse=True)[:3]
                
                # Agregar información adicional a cada coincidencia
                for coincidencia in mejores_coincidencias:
                    coincidencia['archivo_problematico'] = archivo_objetivo.name
                
                resultados.extend(mejores_coincidencias)
                
                # MOSTRAR RESULTADOS EN TIEMPO REAL
                tqdm.write("🎯 MEJORES COINCIDENCIAS ENCONTRADAS:")
                tqdm.write("-" * 60)
                for j, coincidencia in enumerate(mejores_coincidencias, 1):
                    icono = "🥇" if j == 1 else "🥈" if j == 2 else "🥉"
                    
                    tqdm.write(f"{icono} #{j} 💾 {coincidencia['similitud']:.1%} - {coincidencia['archivo']}")
                    tqdm.write(f"    📄 Páginas: {coincidencia['paginas']}")
                        
                    if parar_en_100 and coincidencia['similitud'] >= 0.999:
                        tqdm.write("    🎯 ¡COINCIDENCIA PERFECTA! Búsqueda detenida.")
                        break
                        
            else:
                # Sin coincidencias
                tqdm.write("❌ Sin coincidencias encontradas")
                reader_objetivo = PdfReader(archivo_objetivo)
                num_paginas_objetivo = len(reader_objetivo.pages)
                
                resultados.append({
                    'archivo_problematico': archivo_objetivo.name,
                    'archivo': 'SIN_COINCIDENCIAS',
                    'ruta': '',
                    'tipo': 'ninguno',
                    'similitud': 0.0,
                    'paginas': num_paginas_objetivo
                })
                
        except Exception as e:
            tqdm.write(f"❌ Error procesando {archivo_objetivo.name}: {e}")
            continue
    
    # Crear DataFrame
    df_resultados = pd.DataFrame(resultados)
    
    if df_resultados.empty:
        print("\n❌ No se encontraron resultados")
        return df_resultados
    
    # Extraer nombre del alumno del archivo problemático
    df_resultados['alumno'] = df_resultados['archivo_problematico'].str.replace('.pdf', '').str.replace('_P[0-9]_.*', '', regex=True)
    
    # Ordenar por alumno y similitud (descendente)
    df_resultados = df_resultados.sort_values(['alumno', 'similitud'], ascending=[True, False])
    
    # Reordenar columnas para mejor visualización
    columnas_orden = ['alumno', 'archivo_problematico', 'similitud', 'archivo', 'tipo', 'paginas', 'ruta']
    
    df_resultados = df_resultados[columnas_orden].reset_index(drop=True)
    
    # RESUMEN FINAL
    print("\n" + "="*80)
    print("📊 RESUMEN FINAL DEL ANÁLISIS")
    print("="*80)
    
    total_archivos = len(archivos_problematicos)
    archivos_con_coincidencias = len(df_resultados[df_resultados['similitud'] > 0])
    archivos_sin_coincidencias = len(df_resultados[df_resultados['similitud'] == 0])
    
    print(f"📁 Total de archivos analizados: {total_archivos}")
    print(f"✅ Archivos con coincidencias: {archivos_con_coincidencias}")
    print(f"❌ Archivos sin coincidencias: {archivos_sin_coincidencias}")
    
    # Estadísticas de similitud
    if archivos_con_coincidencias > 0:
        coincidencias_df = df_resultados[df_resultados['similitud'] > 0]
        similitud_promedio = coincidencias_df['similitud'].mean()
        similitud_maxima = coincidencias_df['similitud'].max()
        
        print(f"📈 Similitud promedio: {similitud_promedio:.1%}")
        print(f"🎯 Similitud máxima: {similitud_maxima:.1%}")
        
        # Solo comparamos con saved, así que toda coincidencia es de tipo saved
        print(f"\n💾 Todas las coincidencias son de /saved: {archivos_con_coincidencias} archivos")
    
    return df_resultados

def comparar_pdfs_visuales_con_tqdm_solo_saved(nombre_archivo_objetivo, 
                                               ruta_saved="../data/saved/", 
                                               ruta_examenes="../data/examenes_procesados/problemático/",
                                               mostrar_detalles=False, 
                                               parar_en_100=False,
                                               umbral_similitud=0.85):
    """
    Versión modificada de comparar_pdfs_visuales_con_tqdm que SOLO compara con /saved
    Sin comparar con lotes para optimizar el análisis de problemáticos
    """
    
    # Buscar el archivo objetivo en la ruta especificada
    archivo_objetivo = None
    ruta_examenes = Path(ruta_examenes)
    
    for archivo_pdf in ruta_examenes.glob("*.pdf"):
        if nombre_archivo_objetivo.lower() in archivo_pdf.name.lower():
            archivo_objetivo = archivo_pdf
            break
    
    if not archivo_objetivo or not archivo_objetivo.exists():
        if mostrar_detalles:
            print(f"❌ No se encontró el archivo {nombre_archivo_objetivo}")
        return None
    
    if mostrar_detalles:
        print(f"🎯 Archivo encontrado: {archivo_objetivo}")
    
    # Obtener número de páginas del objetivo
    try:
        reader_objetivo = PdfReader(archivo_objetivo)
        num_paginas_objetivo = len(reader_objetivo.pages)
        if mostrar_detalles:
            print(f"📄 Páginas del archivo objetivo: {num_paginas_objetivo}")
    except Exception as e:
        if mostrar_detalles:
            print(f"❌ Error leyendo archivo objetivo: {e}")
        return None
    
    # Convertir páginas del objetivo a imágenes para comparación
    try:
        imagenes_objetivo = convert_from_path(
            archivo_objetivo, 
            dpi=100,
            fmt='jpeg'
        )
    except Exception as e:
        if mostrar_detalles:
            print(f"❌ Error convirtiendo archivo objetivo: {e}")
        return None
    
    # Lista para almacenar similitudes
    coincidencias_encontradas = []
    
    # COMPARAR SOLO CON ARCHIVOS EN /saved
    ruta_saved = Path(ruta_saved)
    
    if ruta_saved.exists():
        archivos_saved = list(ruta_saved.glob("*.pdf"))
        
        # Usar tqdm para mostrar progreso
        progress_bar = tqdm(archivos_saved, desc="💾 Comparando con /saved", leave=False, unit="archivo")
        
        for archivo_saved in progress_bar:
            try:
                reader_saved = PdfReader(archivo_saved)
                num_paginas_saved = len(reader_saved.pages)
                
                # Solo comparar si tienen el mismo número de páginas
                if num_paginas_saved == num_paginas_objetivo:
                    # Convertir a imágenes
                    imagenes_saved = convert_from_path(archivo_saved, dpi=100, fmt='jpeg')
                    
                    # Comparar cada página
                    similitud_total = comparar_imagenes_paginas(imagenes_objetivo, imagenes_saved)
                    
                    if similitud_total >= umbral_similitud:  # Usar umbral personalizado
                        similitud_info = {
                            'archivo': archivo_saved.name,
                            'ruta': str(archivo_saved),
                            'tipo': 'saved',
                            'similitud': similitud_total,
                            'paginas': num_paginas_saved
                        }
                        coincidencias_encontradas.append(similitud_info)
                        
                        if mostrar_detalles:
                            tqdm.write(f"    ✅ COINCIDENCIA: {similitud_total:.2%}")
                        
                        # PARADA AUTOMÁTICA al 100%
                        if parar_en_100 and similitud_total >= 0.999:
                            if mostrar_detalles:
                                tqdm.write(f"    🎯 ¡100% SIMILITUD ENCONTRADA! Deteniendo búsqueda...")
                            break
                        
            except Exception as e:
                if mostrar_detalles:
                    tqdm.write(f"  ❌ Error procesando {archivo_saved.name}: {e}")
                continue
        
        progress_bar.close()
    
    # Retornar coincidencias ordenadas por similitud
    if coincidencias_encontradas:
        coincidencias_encontradas.sort(key=lambda x: x['similitud'], reverse=True)
    
    return coincidencias_encontradas

In [ ]:

# LLAMADAS DE EJEMPLO:

# 1. Análisis RÁPIDO con parada automática al 100%
print("🚀 Iniciando análisis RÁPIDO con parada automática...")
df_problematicos_rapido = analizar_examenes_problematicos(
    parar_en_100=True,  # ⭐ PARADA AUTOMÁTICA ACTIVADA
    umbral_similitud=0.8  # Umbral más alto para ser más selectivo

🚀 Iniciando análisis RÁPIDO con parada automática...
🔍 Analizando 15 archivos problemáticos...
💾 Solo comparando con archivos en /saved
⏹️ Modo parada automática activado (se detiene al encontrar 100% similitud)


📄 Procesando archivos:   0%|          | 0/15 [00:00<?, ?archivo/s]


📄 Procesando: CAMARA_VILKOVA_VERONICA_LUISA_P3_IWSIM11.pdf


📄 Procesando archivos:   7%|▋         | 1/15 [00:06<01:26,  6.16s/archivo]

🎯 MEJORES COINCIDENCIAS ENCONTRADAS:
------------------------------------------------------------
🥇 #1 💾 100.0% - lote_13_examen_15.pdf
    📄 Páginas: 1
    🎯 ¡COINCIDENCIA PERFECTA! Búsqueda detenida.

📄 Procesando: FUENTE_MARTINEZ_HERNAN_GABRIEL_DE_LA_P5_IWSIM12.pdf


...

📄 Procesando archivos:  93%|█████████▎| 14/15 [02:59<00:11, 11.42s/archivo]

🎯 MEJORES COINCIDENCIAS ENCONTRADAS:
------------------------------------------------------------
🥇 #1 💾 100.0% - lote_3_examen_27.pdf
    📄 Páginas: 1
    🎯 ¡COINCIDENCIA PERFECTA! Búsqueda detenida.

📄 Procesando: ZODER_MENDEZ_PABLO_JOACHIM_P5_IWSIM12.pdf


📄 Procesando archivos: 100%|██████████| 15/15 [03:04<00:00, 12.30s/archivo]

🎯 MEJORES COINCIDENCIAS ENCONTRADAS:
------------------------------------------------------------
🥇 #1 💾 100.0% - lote_12_examen_5.pdf
    📄 Páginas: 1
    🎯 ¡COINCIDENCIA PERFECTA! Búsqueda detenida.

📊 RESUMEN FINAL DEL ANÁLISIS
📁 Total de archivos analizados: 15
✅ Archivos con coincidencias: 44
❌ Archivos sin coincidencias: 0
📈 Similitud promedio: 96.4%
🎯 Similitud máxima: 100.0%

💾 Todas las coincidencias son de /saved: 44 archivos


NameError: name 'p' is not defined

In [95]:
import os
import shutil
from pathlib import Path
from pypdf import PdfReader, PdfWriter
import re

def separar_archivo_conjunto_y_mover(archivo_path, carpeta_base="../data/examenes_procesados/"):
    """
    Separa un archivo PDF conjunto en dos archivos individuales y los mueve a sus carpetas correspondientes
    """
    archivo_path = Path(archivo_path)
    
    if not archivo_path.exists():
        print(f"❌ El archivo {archivo_path} no existe")
        return
    
    # Extraer información del nombre del archivo
    nombre_archivo = archivo_path.stem
    
    # Buscar el patrón _Y_ para separar los nombres
    if '_Y_' not in nombre_archivo:
        print(f"❌ El archivo no contiene el patrón '_Y_' para separar")
        return
    
    # Separar por _Y_
    partes = nombre_archivo.split('_Y_')
    if len(partes) != 2:
        print(f"❌ El archivo no tiene exactamente dos partes separadas por '_Y_'")
        return
    
    primer_alumno = partes[0].strip()
    segundo_alumno = partes[1].strip()
    
    print(f"📄 Procesando archivo: {archivo_path.name}")
    print(f"👤 Primer alumno: {primer_alumno}")
    print(f"👤 Segundo alumno: {segundo_alumno}")
    
    # Leer el PDF original
    try:
        reader = PdfReader(archivo_path)
        total_paginas = len(reader.pages)
        print(f"📊 Total de páginas en el archivo: {total_paginas}")
        
        if total_paginas != 4:
            print(f"⚠️ El archivo tiene {total_paginas} páginas, se esperaban 4 para dividir en dos archivos de 2 páginas cada uno")
        
        # Crear primer archivo (páginas 1-2)
        writer1 = PdfWriter()
        for i in range(min(2, total_paginas)):
            writer1.add_page(reader.pages[i])
        
        # Crear segundo archivo (páginas 3-4)
        writer2 = PdfWriter()
        for i in range(2, min(4, total_paginas)):
            writer2.add_page(reader.pages[i])
        
        # Determinar rutas de destino basándose en los nombres
        carpeta_base = Path(carpeta_base)
        
        # Extraer información del primer alumno
        grupo1, practica1 = extraer_grupo_y_practica(primer_alumno)
        archivo1_nombre = f"{primer_alumno}.pdf"
        
        if grupo1 != "desconocido":
            carpeta1 = carpeta_base / grupo1 / f"Practica_{practica1}"
        else:
            carpeta1 = carpeta_base / "problemático"
        
        # Extraer información del segundo alumno  
        grupo2, practica2 = extraer_grupo_y_practica(segundo_alumno)
        archivo2_nombre = f"{segundo_alumno}.pdf"
        
        if grupo2 != "desconocido":
            carpeta2 = carpeta_base / grupo2 / f"Practica_{practica2}"
        else:
            carpeta2 = carpeta_base / "problemático"
        
        # Crear carpetas si no existen
        carpeta1.mkdir(parents=True, exist_ok=True)
        carpeta2.mkdir(parents=True, exist_ok=True)
        
        # Guardar archivos
        archivo1_path = carpeta1 / archivo1_nombre
        archivo2_path = carpeta2 / archivo2_nombre
        
        # Evitar conflictos de nombres
        contador1 = 2
        while archivo1_path.exists():
            nombre_base1 = primer_alumno
            archivo1_path = carpeta1 / f"{nombre_base1}_{contador1}.pdf"
            contador1 += 1
        
        contador2 = 2
        while archivo2_path.exists():
            nombre_base2 = segundo_alumno
            archivo2_path = carpeta2 / f"{nombre_base2}_{contador2}.pdf"
            contador2 += 1
        
        # Escribir archivos
        with open(archivo1_path, 'wb') as f1:
            writer1.write(f1)
        
        with open(archivo2_path, 'wb') as f2:
            writer2.write(f2)
        
        print(f"\n✅ ARCHIVOS CREADOS EXITOSAMENTE:")
        print(f"📁 Archivo 1: {archivo1_path}")
        print(f"   📄 Páginas: 1-2")
        print(f"   👤 Alumno: {primer_alumno}")
        print(f"   🎯 Grupo: {grupo1}, Práctica: {practica1}")
        
        print(f"\n📁 Archivo 2: {archivo2_path}")
        print(f"   📄 Páginas: 3-4 (del original)")
        print(f"   👤 Alumno: {segundo_alumno}")
        print(f"   🎯 Grupo: {grupo2}, Práctica: {practica2}")
        
        # Preguntar si eliminar el archivo original
        print(f"\n🗑️ ¿Deseas eliminar el archivo original?")
        print(f"   Archivo: {archivo_path}")
        eliminar = input("   Escribe 'si' para eliminarlo: ").lower().strip()
        
        if eliminar == 'si':
            archivo_path.unlink()
            print(f"   ✅ Archivo original eliminado")
        else:
            print(f"   📄 Archivo original conservado")
        
        return [archivo1_path, archivo2_path]
        
    except Exception as e:
        print(f"❌ Error procesando el archivo: {e}")
        return None

def extraer_grupo_y_practica(nombre_alumno):
    """
    Extrae el grupo y práctica del nombre del alumno
    """
    
    # Buscar práctica (P seguido de número)
    match_practica = re.search(r'P(\d+)', nombre_alumno)
    if match_practica:
        practica = match_practica.group(1)
    else:
        practica = "3"  # Por defecto
    
    # Buscar grupo (patrones conocidos)
    grupos_posibles = ['IWSIM11', 'IWSIM12', 'IWSIT11', 'IWSIT12', 'CITIM11', 'CITIM12', 'CITIT11', 'CITIT12']
    
    for grupo in grupos_posibles:
        if grupo in nombre_alumno:
            return grupo, practica
    
    return "desconocido", practica

## Ejecutar la separación
#archivo_a_separar = "..."
#archivos_resultantes = separar_archivo_conjunto_y_mover(archivo_a_separar)

#if archivos_resultantes:
    #print(f"\n🎉 ¡Proceso completado exitosamente!")
    #print(f"📊 Se crearon {len(archivos_resultantes)} archivos nuevos")